In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime as dt
import warnings
from geopy.distance import geodesic
import matplotlib.pyplot as plt
import itertools
warnings.filterwarnings("ignore")

In [2]:
df =pd.read_csv("CICFinalDataSetWithCICSummaryData20240809_v2.csv")

In [3]:
df = df[df['LoanProduct'] == 'SIL-Instore']
# df = df[df['LoanProduct'].isna() == False]

In [4]:
df_table_A = df[['digitalLoanAccountId','customerId','disbursementDateTime','targetdataselection_SIL','defFPD30']]

# Filter data

In [5]:
df['SanctionedAmount'] = df['CreditLimit'].fillna(df['FinancedAmount'])

df['disbursementDateTime'] = pd.to_datetime(df['disbursementDateTime']).dt.date
df['ContractStartDate'] = pd.to_datetime(df['ContractStartDate']).dt.date
df['ContractEndDate'] = pd.to_datetime(df['ContractEndDate'], errors='coerce')
df['run_date'] = pd.to_datetime(df['run_date']).dt.date
df['ContractRequestDate'] = pd.to_datetime(df['ContractRequestDate']).dt.date

df['ContractPhaseDesc'] = np.where((df['ContractEndDate'].notnull()) & (df['ContractEndDate'] < df['run_date']),
                                    'Closed',df['ContractPhaseDesc'])


df['OverdueDaysDesc'] = np.where((df['OverduePaymentsAmount'] == 0) & (df['OverduePaymentsNumber'] == 0),'Paid as agreed / Current',
                                                            df['OverdueDaysDesc'])

df['OverdueDays'] = np.where((df['OverduePaymentsAmount'] == 0) & (df['OverduePaymentsNumber'] == 0),0,  df['OverdueDays'])

In [6]:
df['RoleDesc'].value_counts()

RoleDesc
Borrower       118765
Co-Borrower       599
Name: count, dtype: int64

# Snapshot Features

## Active Contract cnt

In [7]:
granted_loans = df[df['SOURCE']=='granted']

granted_loans['disbursementDateTime'] = pd.to_datetime(granted_loans['disbursementDateTime']).dt.date
granted_loans['ContractStartDate'] = pd.to_datetime(granted_loans['ContractStartDate']).dt.date

In [8]:
snapshot_granted_loans = granted_loans[granted_loans['run_date'] >= granted_loans['ContractStartDate']]
active_contract_loans = snapshot_granted_loans[snapshot_granted_loans['ContractPhaseDesc'] =='Active']

In [9]:
active_contract_loans_count_df =active_contract_loans.groupby(['digitalLoanAccountId']).agg(
                                         cnt_active_contracts=('CBContractCode', 'nunique')).reset_index()
active_contract_loans_count_df.head(5)

,digitalLoanAccountId,cnt_active_contracts
0,000a3bee-c57d-4647-beaa-f8d3293b4101,1
1,000d5b99-ffd3-45ad-b650-32fe43a95dc7,19
2,001427de-8ac2-4a4e-aabb-40942c743276,1
3,001afde5-17dd-48c2-9693-b6df1aa8e752,1
4,0021596e-dcda-4634-838f-9475aef54834,1


## Active contracts by loan segment

In [10]:
active_contract_loans_by_loansegment =active_contract_loans.groupby(['digitalLoanAccountId','loan_segment']).agg(
                                         cnt_active_contracts=('CBContractCode', 'nunique')).reset_index()

pivot_active_contract_loans_by_loansegment = active_contract_loans_by_loansegment.pivot_table(index=['digitalLoanAccountId'], columns='loan_segment', values=['cnt_active_contracts'])

pivot_active_contract_loans_by_loansegment.columns = [metric+'_'+sector for metric, sector in pivot_active_contract_loans_by_loansegment.columns]
pivot_active_contract_loans_by_loansegment = pivot_active_contract_loans_by_loansegment.reset_index()

pivot_active_contract_loans_by_loansegment.head(5)

,digitalLoanAccountId,cnt_active_contracts_Credit Cards,cnt_active_contracts_Credit Lines,cnt_active_contracts_Home Equity Loans,cnt_active_contracts_Other Loans,cnt_active_contracts_Personal Loans,cnt_active_contracts_Real Estate Loans,cnt_active_contracts_Short and Term Loans,cnt_active_contracts_Time Loans
0,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1,000d5b99-ffd3-45ad-b650-32fe43a95dc7,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,001427de-8ac2-4a4e-aabb-40942c743276,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,001afde5-17dd-48c2-9693-b6df1aa8e752,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,0021596e-dcda-4634-838f-9475aef54834,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


## Active contract max limit (max sanctioned amt)

In [11]:
max_creditlimit_currently_activeloans  = active_contract_loans.groupby(['digitalLoanAccountId']).agg(
                                         max_amt_active_contracts=('SanctionedAmount', 'max')).reset_index()
max_creditlimit_currently_activeloans.head(5)

,digitalLoanAccountId,max_amt_active_contracts
0,000a3bee-c57d-4647-beaa-f8d3293b4101,67400.0
1,000d5b99-ffd3-45ad-b650-32fe43a95dc7,167000.0
2,001427de-8ac2-4a4e-aabb-40942c743276,20000.0
3,001afde5-17dd-48c2-9693-b6df1aa8e752,9000.0
4,0021596e-dcda-4634-838f-9475aef54834,29000.0


## Active contract total outstanding balance

In [12]:
total_OutstandingBal_currently_activeloans  =active_contract_loans.groupby(['digitalLoanAccountId']).agg(
                                         tot_active_contracts_os_amt=('OutstandingBalance', 'sum')).reset_index()

total_OutstandingBal_currently_activeloans.head(5)

,digitalLoanAccountId,tot_active_contracts_os_amt
0,000a3bee-c57d-4647-beaa-f8d3293b4101,44049.0
1,000d5b99-ffd3-45ad-b650-32fe43a95dc7,703979.0
2,001427de-8ac2-4a4e-aabb-40942c743276,0.0
3,001afde5-17dd-48c2-9693-b6df1aa8e752,6839.0
4,0021596e-dcda-4634-838f-9475aef54834,17280.0


## Overall Utilization

In [13]:
total_Utilization_currently_activeloans  =active_contract_loans.groupby(['digitalLoanAccountId']).agg(
                                         tot_outstandingbal_amt=('OutstandingBalance', 'sum'),
                                          tot_Sanctioned_amt=('SanctionedAmount', 'sum')  ).reset_index()


In [14]:

# Calculate the ratio of sum(Outstanding) to sum(FinancedAmount) for each group
total_Utilization_currently_activeloans['tot_active_contracts_util'] = total_Utilization_currently_activeloans['tot_outstandingbal_amt'] / total_Utilization_currently_activeloans['tot_Sanctioned_amt']

total_Utilization_currently_activeloans.replace([np.inf, -np.inf], -1, inplace=True)
total_Utilization_currently_activeloans = total_Utilization_currently_activeloans[['digitalLoanAccountId','tot_active_contracts_util']]
total_Utilization_currently_activeloans.head()

,digitalLoanAccountId,tot_active_contracts_util
0,000a3bee-c57d-4647-beaa-f8d3293b4101,0.653546
1,000d5b99-ffd3-45ad-b650-32fe43a95dc7,0.536569
2,001427de-8ac2-4a4e-aabb-40942c743276,0.000000
3,001afde5-17dd-48c2-9693-b6df1aa8e752,0.759889
4,0021596e-dcda-4634-838f-9475aef54834,0.595862


In [15]:
total_Utilization_currently_activeloans[total_Utilization_currently_activeloans['tot_active_contracts_util']>=2]

,digitalLoanAccountId,tot_active_contracts_util
8,004e0430-0f52-4db7-a12d-86c3fa10440a,2.155334
137,03981251-65b3-41ee-9dde-3e7cb20cd1a8,2.047264
214,051ab7d3-cd85-4c5c-bc49-fdef82c1ec87,10.015887
242,05d43b8a-b642-4a75-804a-cc93c90bbaac,2.128800
532,0c94730a-b396-41e3-8b88-7583d025e709,9.599289
...,...,...
10457,f6522142-85c9-46e4-9bd3-9c1a24793fde,2.542836
10536,f861a01a-9227-44f8-8972-06834b2a04dd,3.486492
10561,f8ede513-3037-45ef-957c-0968110a7c6a,2.121251
10685,fb9ff6f6-06c8-414a-b888-5fa713daa7ab,2.126042


In [16]:
active_contract_loans[active_contract_loans['digitalLoanAccountId'] == '004e0430-0f52-4db7-a12d-86c3fa10440a'][['OutstandingBalance','SanctionedAmount','FinancedAmount','CreditLimit']]

,OutstandingBalance,SanctionedAmount,FinancedAmount,CreditLimit
90995,165840.0,76944.0,76944.0,NaN


In [17]:
active_contract_loans[active_contract_loans['digitalLoanAccountId'] == 'e3ef5a12-38d2-4224-8d5a-ef292d56f611'][['OutstandingBalance','SanctionedAmount','FinancedAmount','CreditLimit']]

,OutstandingBalance,SanctionedAmount,FinancedAmount,CreditLimit
50988,175787.0,4883.0,4883.0,NaN


In [18]:
active_contract_loans[active_contract_loans['digitalLoanAccountId'] == 'ae577f39-28e0-4efa-8844-15b7e2dde91c'][['OutstandingBalance','SanctionedAmount','FinancedAmount','CreditLimit']] 

,OutstandingBalance,SanctionedAmount,FinancedAmount,CreditLimit
38314,84429.0,3127.0,3127.0,NaN
102912,15362.0,50000.0,50000.0,NaN


In [19]:
active_contract_loans[active_contract_loans['digitalLoanAccountId'] == '150f1383-db34-4f7c-80ee-1d5452ab7f72'][['OutstandingBalance','SanctionedAmount','FinancedAmount','CreditLimit']] 
 

,OutstandingBalance,SanctionedAmount,FinancedAmount,CreditLimit
2702,94468.0,66500.0,66500.0,NaN
17319,121694.0,0.0,NaN,0.0
202017,105758.0,0.0,NaN,0.0


## Overall Utilization by loan segment

In [20]:
total_Utilization_by_seg =active_contract_loans.groupby(['digitalLoanAccountId','loan_segment']).agg(
                                         tot_outstandingbal_amt=('OutstandingBalance', 'sum'),
                                          tot_Sanctioned_amt=('SanctionedAmount', 'sum')  ).reset_index()

total_Utilization_by_seg['tot_active_contracts_util'] = total_Utilization_by_seg['tot_outstandingbal_amt'] / total_Utilization_by_seg['tot_Sanctioned_amt']


In [21]:
total_Utilization_by_seg = total_Utilization_by_seg[['digitalLoanAccountId','loan_segment','tot_active_contracts_util']]

pivot_atotal_Utilization_by_seg = total_Utilization_by_seg.pivot_table(index=['digitalLoanAccountId'], columns='loan_segment', values=['tot_active_contracts_util'])

pivot_atotal_Utilization_by_seg.columns = [segment+'_'+'active_contracts_util' for metric, segment in pivot_atotal_Utilization_by_seg.columns]
pivot_atotal_Utilization_by_seg = pivot_atotal_Utilization_by_seg.reset_index()

pivot_atotal_Utilization_by_seg.head(5)

,digitalLoanAccountId,Credit Cards_active_contracts_util,Credit Lines_active_contracts_util,Home Equity Loans_active_contracts_util,Other Loans_active_contracts_util,Personal Loans_active_contracts_util,Real Estate Loans_active_contracts_util,Short and Term Loans_active_contracts_util,Time Loans_active_contracts_util
0,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,NaN,NaN,0.653546,NaN,NaN,NaN,NaN
1,000d5b99-ffd3-45ad-b650-32fe43a95dc7,0.536569,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,001427de-8ac2-4a4e-aabb-40942c743276,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,001afde5-17dd-48c2-9693-b6df1aa8e752,NaN,NaN,NaN,NaN,0.759889,NaN,NaN,NaN
4,0021596e-dcda-4634-838f-9475aef54834,NaN,NaN,NaN,NaN,0.595862,NaN,NaN,NaN


## Last repayment amount

In [22]:
granted_loans = df[df['SOURCE'] == 'granted']
 

granted_loans['LastPaymentDate'] = pd.to_datetime(granted_loans['LastPaymentDate'], errors='coerce')

In [23]:
LastPaymentDate = granted_loans[(granted_loans['LastPaymentDate'] <= granted_loans['run_date'])  ]               
                                
latest_repayment_date = LastPaymentDate.groupby('digitalLoanAccountId')['LastPaymentDate'].max().reset_index()

latest_repayment = latest_repayment_date.merge(
                        LastPaymentDate[['digitalLoanAccountId', 'LastPaymentDate', 'LastPaymentAmount','PaymentMethodDesc']], on=['digitalLoanAccountId', 'LastPaymentDate'], how='left')
 
last_repay_amt_ = latest_repayment.groupby('digitalLoanAccountId').agg(
                                         last_repay_amt=('LastPaymentAmount', 'sum')).reset_index()


last_repay_amt = latest_repayment.loc[latest_repayment.groupby('digitalLoanAccountId')['LastPaymentAmount'].idxmax()]
last_repay_amt.rename(columns={'PaymentMethodDesc': 'highest_latest_repay_method'}, inplace=True)


last_repay_amt = pd.merge(last_repay_amt_, last_repay_amt[['digitalLoanAccountId','highest_latest_repay_method']], on='digitalLoanAccountId', how='left')

last_repay_amt_all = last_repay_amt[['digitalLoanAccountId','last_repay_amt', 'highest_latest_repay_method']]
last_repay_amt_all.head()

,digitalLoanAccountId,last_repay_amt,highest_latest_repay_method
0,000a3bee-c57d-4647-beaa-f8d3293b4101,4726.0,NaN
1,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,253.0,Other
2,000d5b99-ffd3-45ad-b650-32fe43a95dc7,7000.0,Cash
3,0010fa50-dedd-471b-93b8-7e2cabf2b71f,2676.0,NaN
4,00122bff-b647-49c5-b993-02fd6518b171,2320.0,NaN


In [24]:
LastPaymentDate_30d = granted_loans[(granted_loans['LastPaymentDate'] >= (granted_loans['run_date'] - pd.Timedelta(days=30))) & 
                 (granted_loans['LastPaymentDate'] <= granted_loans['run_date'])]

# Get the latest repayment date per digitalLoanAccountId
latest_repayment_date_30d = LastPaymentDate_30d.groupby('digitalLoanAccountId')['LastPaymentDate'].max().reset_index()

# Merge to get the corresponding RepaymentAmount
latest_repayment_30d = latest_repayment_date_30d.merge(
                        LastPaymentDate_30d[['digitalLoanAccountId', 'LastPaymentDate', 'LastPaymentAmount','PaymentMethodDesc']], on=['digitalLoanAccountId', 'LastPaymentDate'], how='left')


latest_repayment_30d_ = latest_repayment_30d.groupby(['digitalLoanAccountId']).agg(
                                         last_repay_amt_30d=('LastPaymentAmount', 'sum') ).reset_index()

 
last_repay_amt_30d = latest_repayment_30d.loc[latest_repayment_30d.groupby('digitalLoanAccountId')['LastPaymentAmount'].idxmax()]
last_repay_amt_30d.rename(columns={'PaymentMethodDesc': 'highest_latest_repay_method_30d'}, inplace=True)


last_repay_amt_30d = pd.merge(latest_repayment_30d_, last_repay_amt_30d[['digitalLoanAccountId','highest_latest_repay_method_30d']], on='digitalLoanAccountId', how='left')

last_repay_amt_30d = last_repay_amt_30d[['digitalLoanAccountId','last_repay_amt_30d', 'highest_latest_repay_method_30d']]
last_repay_amt_30d.head()

,digitalLoanAccountId,last_repay_amt_30d,highest_latest_repay_method_30d
0,000d5b99-ffd3-45ad-b650-32fe43a95dc7,7000.0,Cash
1,01c8b702-72cf-4792-8e78-713176eb46f0,0.0,Other
2,02b9d3ce-ed74-4c8b-956b-e2578a440313,1659.0,Other
3,0437cbce-b34d-4c66-a576-f7fcf7663089,676.0,Cash
4,04748279-7541-4a03-904e-17f194d0ea17,0.0,Other


In [25]:
LastPaymentDate_60d = granted_loans[(granted_loans['LastPaymentDate'] >= (granted_loans['run_date'] - pd.Timedelta(days=60))) & 
                 (granted_loans['LastPaymentDate'] <= granted_loans['run_date'])]

# Get the latest repayment date per digitalLoanAccountId
latest_repayment_date_60d = LastPaymentDate_60d.groupby('digitalLoanAccountId')['LastPaymentDate'].max().reset_index()

# Merge to get the corresponding RepaymentAmount
latest_repayment_60d = latest_repayment_date_60d.merge(
                        LastPaymentDate_60d[['digitalLoanAccountId', 'LastPaymentDate', 'LastPaymentAmount','PaymentMethodDesc']], on=['digitalLoanAccountId', 'LastPaymentDate'], how='left')
 
latest_repayment_60d_ = latest_repayment_60d.groupby(['digitalLoanAccountId']).agg(
                                         last_repay_amt_60d=('LastPaymentAmount', 'sum') ).reset_index()
    
last_repay_amt_60d = latest_repayment_60d.loc[latest_repayment_60d.groupby('digitalLoanAccountId')['LastPaymentAmount'].idxmax()]
last_repay_amt_60d.rename(columns={'LastPaymentAmount': 'last_repay_amt_60d','PaymentMethodDesc': 'highest_latest_repay_method_60d'}, inplace=True)


last_repay_amt_60d = pd.merge(latest_repayment_60d_, last_repay_amt_60d[['digitalLoanAccountId','highest_latest_repay_method_60d']], on='digitalLoanAccountId', how='left')


last_repay_amt_60d = last_repay_amt_60d[['digitalLoanAccountId','last_repay_amt_60d', 'highest_latest_repay_method_60d']]
last_repay_amt_60d.head()

,digitalLoanAccountId,last_repay_amt_60d,highest_latest_repay_method_60d
0,000d5b99-ffd3-45ad-b650-32fe43a95dc7,7000.0,Cash
1,001afde5-17dd-48c2-9693-b6df1aa8e752,450.0,Cash
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,1985.0,Current Account Debit
3,0025fd24-406b-475c-bf32-36f93457efc9,2253.0,Current Account Debit
4,0072658b-6a2c-42ad-b087-3fecef73323d,2784.0,Current Account Debit


In [26]:
LastPaymentDate_90d = granted_loans[(granted_loans['LastPaymentDate'] >= (granted_loans['run_date'] - pd.Timedelta(days=90))) & 
                 (granted_loans['LastPaymentDate'] <= granted_loans['run_date'])]

# Get the latest repayment date per digitalLoanAccountId
latest_repayment_date_90d = LastPaymentDate_90d.groupby('digitalLoanAccountId')['LastPaymentDate'].max().reset_index()

# Merge to get the corresponding RepaymentAmount
latest_repayment_90d = latest_repayment_date_90d.merge(
                        LastPaymentDate_90d[['digitalLoanAccountId', 'LastPaymentDate', 'LastPaymentAmount','PaymentMethodDesc']], on=['digitalLoanAccountId', 'LastPaymentDate'], how='left')

latest_repayment_90d_ = latest_repayment_90d.groupby(['digitalLoanAccountId']).agg(
                                         last_repay_amt_90d=('LastPaymentAmount', 'sum') ).reset_index()


last_repay_amt_90d = latest_repayment_90d.loc[latest_repayment_90d.groupby('digitalLoanAccountId')['LastPaymentAmount'].idxmax()]
last_repay_amt_90d.rename(columns={'LastPaymentAmount': 'last_repay_amt_90d','PaymentMethodDesc': 'highest_latest_repay_method_90d'}, inplace=True)

last_repay_amt_90d = pd.merge(latest_repayment_90d_, last_repay_amt_90d[['digitalLoanAccountId','highest_latest_repay_method_90d']], on='digitalLoanAccountId', how='left')


last_repay_amt_90d = last_repay_amt_90d[['digitalLoanAccountId','last_repay_amt_90d', 'highest_latest_repay_method_90d']]
last_repay_amt_90d.head()

,digitalLoanAccountId,last_repay_amt_90d,highest_latest_repay_method_90d
0,000d5b99-ffd3-45ad-b650-32fe43a95dc7,7000.0,Cash
1,001afde5-17dd-48c2-9693-b6df1aa8e752,450.0,Cash
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,1985.0,Current Account Debit
3,0025fd24-406b-475c-bf32-36f93457efc9,2253.0,Current Account Debit
4,004f5fad-6194-4901-821e-6792a549cc89,395.0,Cash


In [27]:
last_repay_amt = last_repay_amt_30d.merge(last_repay_amt_60d, on='digitalLoanAccountId', how ='left')
last_repay_amt = last_repay_amt.merge(last_repay_amt_90d, on='digitalLoanAccountId', how ='left')
last_repay_amt = last_repay_amt.merge(last_repay_amt_all, on='digitalLoanAccountId', how ='left')

In [28]:
last_repay_amt.head()

,digitalLoanAccountId,last_repay_amt_30d,highest_latest_repay_method_30d,last_repay_amt_60d,highest_latest_repay_method_60d,last_repay_amt_90d,highest_latest_repay_method_90d,last_repay_amt,highest_latest_repay_method
0,000d5b99-ffd3-45ad-b650-32fe43a95dc7,7000.0,Cash,7000.0,Cash,7000.0,Cash,7000.0,Cash
1,01c8b702-72cf-4792-8e78-713176eb46f0,0.0,Other,0.0,Other,0.0,Other,0.0,Other
2,02b9d3ce-ed74-4c8b-956b-e2578a440313,1659.0,Other,1659.0,Other,1659.0,Other,1659.0,Other
3,0437cbce-b34d-4c66-a576-f7fcf7663089,676.0,Cash,676.0,Cash,676.0,Cash,676.0,Cash
4,04748279-7541-4a03-904e-17f194d0ea17,0.0,Other,0.0,Other,0.0,Other,0.0,Other


## max_age_active_contracts_snapshot

In [29]:
active_contract_loans['ContractStartDate'] = pd.to_datetime(active_contract_loans['ContractStartDate'], format='%d-%m-%Y', errors='coerce')
active_contract_loans['run_date'] = pd.to_datetime(active_contract_loans['run_date'], format='%d-%m-%Y', errors='coerce')
active_contract_loans['ContractEndDate'] = pd.to_datetime(active_contract_loans['ContractEndDate'], format='%d-%m-%Y', errors='coerce')

In [30]:
active_contract_loans['date_diff'] = np.where(
                        active_contract_loans['ContractEndDate'].notna() & (active_contract_loans['run_date'] >= active_contract_loans['ContractEndDate']),
                        (active_contract_loans['ContractEndDate'] - active_contract_loans['ContractStartDate']).dt.days,
                        (active_contract_loans['run_date'] - active_contract_loans['ContractStartDate']).dt.days)


# Find the max date difference for each Loan_id
max_age_active_loans = active_contract_loans.groupby('digitalLoanAccountId').agg(
                                        max_age_active_contracts_snapshot=('date_diff', 'max')).reset_index()

In [31]:
max_age_active_loans.head()

,digitalLoanAccountId,max_age_active_contracts_snapshot
0,000a3bee-c57d-4647-beaa-f8d3293b4101,524.0
1,000d5b99-ffd3-45ad-b650-32fe43a95dc7,2710.0
2,001427de-8ac2-4a4e-aabb-40942c743276,584.0
3,001afde5-17dd-48c2-9693-b6df1aa8e752,104.0
4,0021596e-dcda-4634-838f-9475aef54834,947.0


## max_age_all_contracts_snapshot

In [32]:
snapshot_granted_loans['ContractStartDate'] = pd.to_datetime(snapshot_granted_loans['ContractStartDate'], format='%d-%m-%Y', errors='coerce')
snapshot_granted_loans['run_date'] = pd.to_datetime(snapshot_granted_loans['run_date'], format='%d-%m-%Y', errors='coerce')
snapshot_granted_loans['ContractEndDate'] = pd.to_datetime(snapshot_granted_loans['ContractEndDate'], format='%d-%m-%Y', errors='coerce')

In [33]:

snapshot_granted_loans['date_diff'] = np.where(
                        snapshot_granted_loans['ContractEndDate'].notna() & (snapshot_granted_loans['run_date'] >= snapshot_granted_loans['ContractEndDate']),
                        (snapshot_granted_loans['ContractEndDate'] - snapshot_granted_loans['ContractStartDate']).dt.days,
                        (snapshot_granted_loans['run_date'] - snapshot_granted_loans['ContractStartDate']).dt.days)


In [34]:

# Find the max date difference for each Loan_id
max_age_granted_loans = snapshot_granted_loans.groupby('digitalLoanAccountId').agg(
                                        max_age_all_contracts_snapshot=('date_diff', 'max')).reset_index()

max_age_granted_loans.head()

,digitalLoanAccountId,max_age_all_contracts_snapshot
0,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,171.0
1,000a3bee-c57d-4647-beaa-f8d3293b4101,524.0
2,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,168.0
3,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,180.0
4,000d5b99-ffd3-45ad-b650-32fe43a95dc7,2710.0


## Days since last enquiry granted and non granted

In [35]:
df['run_date'] = pd.to_datetime(df['run_date'], errors='coerce')
df['ContractStartDate'] = pd.to_datetime(df['ContractStartDate'], errors='coerce')
df['ContractRequestDate'] = pd.to_datetime(df['ContractRequestDate'], errors='coerce')

df['no_of_last_enquiry_days'] = np.where(df['ContractStartDate'].isna(),
                                (df['run_date'] - df['ContractRequestDate']).dt.days,
                                (df['run_date'] - df['ContractStartDate']).dt.days)

In [36]:
last_enquiry_days = df.groupby('digitalLoanAccountId').agg(
                                        days_since_last_inquiry=('no_of_last_enquiry_days', 'min')).reset_index()
last_enquiry_days.head()

,digitalLoanAccountId,days_since_last_inquiry
0,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,2.0
1,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,1534.0
2,00087e49-c94a-482d-831a-8fd762f28ca5,0.0
3,000a3bee-c57d-4647-beaa-f8d3293b4101,295.0
4,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,370.0


## Days since last grant date

In [37]:


# Find the latest 'ContractStartDate' for each 'digitalLoanAccountId'
latest_grant_dates = snapshot_granted_loans.groupby(['digitalLoanAccountId','run_date'])['ContractStartDate'].max().reset_index()

latest_grant_dates['days_since_last_grant'] = (latest_grant_dates['run_date'] - latest_grant_dates['ContractStartDate']).dt.days
latest_grant_dates = latest_grant_dates[['digitalLoanAccountId','days_since_last_grant']]
latest_grant_dates.head()

,digitalLoanAccountId,days_since_last_grant
0,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,1534
1,000a3bee-c57d-4647-beaa-f8d3293b4101,524
2,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,370
3,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,373
4,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,439


## Days since last closed date

In [38]:
# Filter the DataFrame to include relevant rows
closed_contract_loans = snapshot_granted_loans[snapshot_granted_loans['ContractPhaseDesc'].isin(['Closed', 'Closed in advance'])]

# Find the latest 'ContractEndDate' for each 'digitalLoanAccountId'
latest_closed_dates = closed_contract_loans.groupby(['digitalLoanAccountId','run_date'])['ContractEndDate'].max().reset_index()

latest_closed_dates['days_since_last_closed'] = (latest_closed_dates['run_date'] - latest_closed_dates['ContractEndDate']).dt.days
latest_closed_dates = latest_closed_dates[['digitalLoanAccountId','days_since_last_closed']]
latest_closed_dates.head(3)

,digitalLoanAccountId,days_since_last_closed
0,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,1363.0
1,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,202.0
2,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,205.0


## Days since last repayment date

In [39]:
snapshot_granted_loans['LastPaymentDate'] = pd.to_datetime(snapshot_granted_loans['LastPaymentDate'])

latest_repay_dates = snapshot_granted_loans.groupby(['digitalLoanAccountId','run_date'])['LastPaymentDate'].max().reset_index()
latest_repay_dates['days_since_last_repay'] = (latest_repay_dates['run_date'] - latest_repay_dates['LastPaymentDate']).dt.days
latest_repay_dates = latest_repay_dates[['digitalLoanAccountId','days_since_last_repay']]
latest_repay_dates.head(3)

,digitalLoanAccountId,days_since_last_repay
0,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,NaN
1,000a3bee-c57d-4647-beaa-f8d3293b4101,166.0
2,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,NaN


## Days since rejection date

In [40]:
refused_contract_loans = df[df['ContractPhaseDesc'] == 'Refused']

refused_contract_loans['ContractRequestDate'] = pd.to_datetime(refused_contract_loans['ContractRequestDate'], errors='coerce')
refused_contract_loans['run_date'] = pd.to_datetime(refused_contract_loans['run_date'], errors='coerce')

# Find the latest 'ContractRequestDate' for each 'digitalLoanAccountId'
latest_reject_dates = refused_contract_loans.groupby(['digitalLoanAccountId','run_date'])['ContractRequestDate'].max().reset_index()
latest_reject_dates['days_since_last_reject'] = (latest_reject_dates['run_date'] - latest_reject_dates['ContractRequestDate']).dt.days
latest_reject_dates = latest_reject_dates[['digitalLoanAccountId','days_since_last_reject']]
latest_reject_dates.head()

,digitalLoanAccountId,days_since_last_reject
0,0065031d-463d-43ec-802b-082390a75bff,NaN
1,016d2e0c-192d-4c7c-90d8-08be55cc31a3,636.0
2,02b8005b-35df-4668-95be-e83d84093f88,2079.0
3,03d98c0e-d88d-4988-8be8-44d158221322,2899.0
4,099ddc2e-ac47-4f00-ae5e-adc2cb317242,3711.0


## Days since renounce date

In [41]:
renounced_contract_loans = df[df['ContractPhaseDesc'] == 'Renounced']

renounced_contract_loans['ContractRequestDate'] = pd.to_datetime(renounced_contract_loans['ContractRequestDate'], errors='coerce')
renounced_contract_loans['run_date'] = pd.to_datetime(renounced_contract_loans['run_date'], errors='coerce')

# Find the latest 'ContractRequestDate' for each 'digitalLoanAccountId'
latest_renounced_dates = renounced_contract_loans.groupby(['digitalLoanAccountId','run_date'])['ContractRequestDate'].max().reset_index()
latest_renounced_dates['days_since_last_renounce'] = (latest_renounced_dates['run_date'] - latest_renounced_dates['ContractRequestDate']).dt.days
latest_renounced_dates = latest_renounced_dates[['digitalLoanAccountId','days_since_last_renounce']]
latest_renounced_dates.head()

,digitalLoanAccountId,days_since_last_renounce
0,0026991f-ce1f-431f-bf64-5c06c85c40e8,1038
1,00f71065-c692-48ec-8bb4-a9d8fb527d1f,2088
2,013f7c44-aa3e-48bb-9abe-2dd82d52c7d9,2158
3,0170778c-ad9b-4d99-8eb4-d28e95be9ef5,2558
4,0198fe8d-bad5-469d-bc3c-03615f501a47,1319


In [42]:
snapshot_feature_df = active_contract_loans_count_df.merge(pivot_active_contract_loans_by_loansegment, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(max_creditlimit_currently_activeloans, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(total_OutstandingBal_currently_activeloans, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(total_Utilization_currently_activeloans, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(pivot_atotal_Utilization_by_seg, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(last_repay_amt, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(max_age_active_loans, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(max_age_granted_loans, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(last_enquiry_days, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(latest_grant_dates, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(latest_closed_dates, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(latest_repay_dates, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(latest_reject_dates, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(latest_renounced_dates, on ='digitalLoanAccountId', how ='outer')

In [43]:
snapshot_feature_df.columns

Index(['digitalLoanAccountId', 'cnt_active_contracts',
       'cnt_active_contracts_Credit Cards',
       'cnt_active_contracts_Credit Lines',
       'cnt_active_contracts_Home Equity Loans',
       'cnt_active_contracts_Other Loans',
       'cnt_active_contracts_Personal Loans',
       'cnt_active_contracts_Real Estate Loans',
       'cnt_active_contracts_Short and Term Loans',
       'cnt_active_contracts_Time Loans', 'max_amt_active_contracts',
       'tot_active_contracts_os_amt', 'tot_active_contracts_util',
       'Credit Cards_active_contracts_util',
       'Credit Lines_active_contracts_util',
       'Home Equity Loans_active_contracts_util',
       'Other Loans_active_contracts_util',
       'Personal Loans_active_contracts_util',
       'Real Estate Loans_active_contracts_util',
       'Short and Term Loans_active_contracts_util',
       'Time Loans_active_contracts_util', 'last_repay_amt_30d',
       'highest_latest_repay_method_30d', 'last_repay_amt_60d',
       'highest_la

# Historical Features For Granted Loans

In [44]:
# df['disbursementDateTime'] = pd.to_datetime(df['disbursementDateTime']).dt.date
# df['ContractStartDate'] = pd.to_datetime(df['ContractStartDate']).dt.date

df['threshold_90days'] = df['run_date'] - pd.Timedelta(days=90)
df['threshold_180days'] = df['run_date'] - pd.Timedelta(days=180)
df['threshold_365days'] = df['run_date'] - pd.Timedelta(days=365)
df['threshold_730days'] = df['run_date'] - pd.Timedelta(days=730)



In [ ]:
granted_loans = df[df['SOURCE']=='granted']

granted_loans_90days = granted_loans[(granted_loans['ContractStartDate'] >= granted_loans['threshold_90days']) & (granted_loans['ContractStartDate'] <= granted_loans['run_date'])]
granted_loans_180days = granted_loans[(granted_loans['ContractStartDate'] >= granted_loans['threshold_180days']) & (granted_loans['ContractStartDate'] <= granted_loans['run_date'])]
granted_loans_365days = granted_loans[(granted_loans['ContractStartDate'] >= granted_loans['threshold_365days']) & (granted_loans['ContractStartDate'] <= granted_loans['run_date'])]
granted_loans_730days = granted_loans[(granted_loans['ContractStartDate'] >= granted_loans['threshold_730days']) & (granted_loans['ContractStartDate'] <= granted_loans['run_date'])]


## Count of Granted loans by loan segment

In [46]:
granted_loans_by_segment_6m =granted_loans_180days.groupby(['digitalLoanAccountId','loan_segment']).agg(
                                         granted_contracts_cnt_6M=('CBContractCode', 'nunique')).reset_index()


pivot_granted_loans_by_segment_6m = granted_loans_by_segment_6m.pivot_table(index=['digitalLoanAccountId'], columns='loan_segment', values=['granted_contracts_cnt_6M'])

pivot_granted_loans_by_segment_6m.columns = [segment+'_granted_contracts_cnt_6M' for metric, segment in pivot_granted_loans_by_segment_6m.columns]
pivot_granted_loans_by_segment_6m = pivot_granted_loans_by_segment_6m.reset_index()


pivot_granted_loans_by_segment_6m.head(5)

,digitalLoanAccountId,Credit Cards_granted_contracts_cnt_6M,Credit Lines_granted_contracts_cnt_6M,Home Equity Loans_granted_contracts_cnt_6M,Other Loans_granted_contracts_cnt_6M,Personal Loans_granted_contracts_cnt_6M,Real Estate Loans_granted_contracts_cnt_6M,Short and Term Loans_granted_contracts_cnt_6M
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,NaN,NaN,NaN,NaN,5.0,NaN,NaN
1,001afde5-17dd-48c2-9693-b6df1aa8e752,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,NaN,NaN,NaN,1.0,NaN,NaN,NaN
3,0022f527-0559-43fb-9e2d-65afcffda50b,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,003369dc-95be-4643-a498-0ec8898c6837,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [47]:
granted_loans_by_segment_12m =granted_loans_365days.groupby(['digitalLoanAccountId','loan_segment']).agg(
                                         granted_contracts_cnt_12M=('CBContractCode', 'nunique')).reset_index()


pivot_granted_loans_by_segment_12m = granted_loans_by_segment_12m.pivot_table(index=['digitalLoanAccountId'], columns='loan_segment', values=['granted_contracts_cnt_12M'])

pivot_granted_loans_by_segment_12m.columns = [segment+'_granted_contracts_cnt_12M' for metric, segment in pivot_granted_loans_by_segment_12m.columns]
pivot_granted_loans_by_segment_12m = pivot_granted_loans_by_segment_12m.reset_index()


pivot_granted_loans_by_segment_12m.head(5)

,digitalLoanAccountId,Credit Cards_granted_contracts_cnt_12M,Credit Lines_granted_contracts_cnt_12M,Home Equity Loans_granted_contracts_cnt_12M,Other Loans_granted_contracts_cnt_12M,Personal Loans_granted_contracts_cnt_12M,Real Estate Loans_granted_contracts_cnt_12M,Short and Term Loans_granted_contracts_cnt_12M
0,00105a59-1559-4dad-a281-6408eecc96c1,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,001a8c7d-b132-4a5b-8686-ebfd945d3bad,NaN,NaN,NaN,NaN,9.0,NaN,NaN
2,001afde5-17dd-48c2-9693-b6df1aa8e752,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,001c77ba-72f9-4c44-a08d-4de9db9f0539,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,0022f527-0559-43fb-9e2d-65afcffda50b,NaN,NaN,NaN,1.0,NaN,NaN,NaN


In [48]:
granted_loans_by_segment_24m =granted_loans_730days.groupby(['digitalLoanAccountId','loan_segment']).agg(
                                         granted_contracts_cnt_24M=('CBContractCode', 'nunique')).reset_index()


pivot_granted_loans_by_segment_24m = granted_loans_by_segment_24m.pivot_table(index=['digitalLoanAccountId'], columns='loan_segment', values=['granted_contracts_cnt_24M'])


pivot_granted_loans_by_segment_24m.columns = [segment+'_granted_contracts_cnt_24M' for metric, segment in pivot_granted_loans_by_segment_24m.columns]
pivot_granted_loans_by_segment_24m = pivot_granted_loans_by_segment_24m.reset_index()


pivot_granted_loans_by_segment_24m.head(5)

,digitalLoanAccountId,Credit Cards_granted_contracts_cnt_24M,Credit Lines_granted_contracts_cnt_24M,Home Equity Loans_granted_contracts_cnt_24M,Other Loans_granted_contracts_cnt_24M,Personal Loans_granted_contracts_cnt_24M,Real Estate Loans_granted_contracts_cnt_24M,Short and Term Loans_granted_contracts_cnt_24M,Time Loans_granted_contracts_cnt_24M,Trust Loans_granted_contracts_cnt_24M
0,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
2,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN
3,000d5b99-ffd3-45ad-b650-32fe43a95dc7,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
4,00105a59-1559-4dad-a281-6408eecc96c1,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [49]:
granted_loan_cnt_by_segment = pivot_granted_loans_by_segment_6m.merge(pivot_granted_loans_by_segment_12m, on=['digitalLoanAccountId'], how='outer')
granted_loan_cnt_by_segment = granted_loan_cnt_by_segment.merge(pivot_granted_loans_by_segment_24m, on=['digitalLoanAccountId'], how='outer')

In [50]:
granted_loan_cnt_by_segment.columns

Index(['digitalLoanAccountId', 'Credit Cards_granted_contracts_cnt_6M',
       'Credit Lines_granted_contracts_cnt_6M',
       'Home Equity Loans_granted_contracts_cnt_6M',
       'Other Loans_granted_contracts_cnt_6M',
       'Personal Loans_granted_contracts_cnt_6M',
       'Real Estate Loans_granted_contracts_cnt_6M',
       'Short and Term Loans_granted_contracts_cnt_6M',
       'Credit Cards_granted_contracts_cnt_12M',
       'Credit Lines_granted_contracts_cnt_12M',
       'Home Equity Loans_granted_contracts_cnt_12M',
       'Other Loans_granted_contracts_cnt_12M',
       'Personal Loans_granted_contracts_cnt_12M',
       'Real Estate Loans_granted_contracts_cnt_12M',
       'Short and Term Loans_granted_contracts_cnt_12M',
       'Credit Cards_granted_contracts_cnt_24M',
       'Credit Lines_granted_contracts_cnt_24M',
       'Home Equity Loans_granted_contracts_cnt_24M',
       'Other Loans_granted_contracts_cnt_24M',
       'Personal Loans_granted_contracts_cnt_24M',
       '

## Count of Granted loans 6m,12m,24m

In [51]:
granted_loans_cnt_12m =granted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                         granted_contracts_cnt_12M=('CBContractCode', 'nunique')).reset_index()
granted_loans_cnt_12m.head()

,digitalLoanAccountId,granted_contracts_cnt_12M
0,00105a59-1559-4dad-a281-6408eecc96c1,1
1,001a8c7d-b132-4a5b-8686-ebfd945d3bad,9
2,001afde5-17dd-48c2-9693-b6df1aa8e752,1
3,001c77ba-72f9-4c44-a08d-4de9db9f0539,1
4,0022f527-0559-43fb-9e2d-65afcffda50b,1


In [52]:
granted_loans_cnt_6m =granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                         granted_contracts_cnt_6M=('CBContractCode', 'nunique')).reset_index()
granted_loans_cnt_6m.head()

,digitalLoanAccountId,granted_contracts_cnt_6M
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,5
1,001afde5-17dd-48c2-9693-b6df1aa8e752,1
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,1
3,0022f527-0559-43fb-9e2d-65afcffda50b,1
4,003369dc-95be-4643-a498-0ec8898c6837,1


In [53]:
granted_loans_cnt_24m =granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                         granted_contracts_cnt_24M=('CBContractCode', 'nunique')).reset_index()
granted_loans_cnt_24m.head()

,digitalLoanAccountId,granted_contracts_cnt_24M
0,000a3bee-c57d-4647-beaa-f8d3293b4101,1
1,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,2
2,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,2
3,000d5b99-ffd3-45ad-b650-32fe43a95dc7,2
4,00105a59-1559-4dad-a281-6408eecc96c1,1


In [54]:
# granted_loans_cnt_6m =granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
#                                          granted_contracts_cnt_6m=('CBContractCode', 'nunique')).reset_index()
# granted_loans_cnt_6m.head()

In [55]:
granted_loan_cnt_df = granted_loans_cnt_6m.merge(granted_loans_cnt_12m, on=['digitalLoanAccountId'], how='outer')
granted_loan_cnt_df = granted_loan_cnt_df.merge(granted_loans_cnt_24m, on=['digitalLoanAccountId'], how='outer')
granted_loan_cnt_df.head()

,digitalLoanAccountId,granted_contracts_cnt_6M,granted_contracts_cnt_12M,granted_contracts_cnt_24M
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,5.0,9.0,12
1,001afde5-17dd-48c2-9693-b6df1aa8e752,1.0,1.0,2
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,1.0,1.0,1
3,0022f527-0559-43fb-9e2d-65afcffda50b,1.0,1.0,1
4,003369dc-95be-4643-a498-0ec8898c6837,1.0,1.0,1


## Amt Granted loans by loan segment

In [56]:
granted_loanamt_by_segment_6m =granted_loans_180days.groupby(['digitalLoanAccountId','loan_segment']).agg(
                                         granted_contracts_amt_6M=('SanctionedAmount', 'sum')).reset_index()


pivot_granted_loanamt_by_segment_6m = granted_loanamt_by_segment_6m.pivot_table(index=['digitalLoanAccountId'], columns='loan_segment', values=['granted_contracts_amt_6M'])

pivot_granted_loanamt_by_segment_6m.columns = [segment+'_granted_contracts_amt_6M' for metric, segment in pivot_granted_loanamt_by_segment_6m.columns]
pivot_granted_loanamt_by_segment_6m = pivot_granted_loanamt_by_segment_6m.reset_index()


pivot_granted_loanamt_by_segment_6m.head(5)

,digitalLoanAccountId,Credit Cards_granted_contracts_amt_6M,Credit Lines_granted_contracts_amt_6M,Home Equity Loans_granted_contracts_amt_6M,Other Loans_granted_contracts_amt_6M,Personal Loans_granted_contracts_amt_6M,Real Estate Loans_granted_contracts_amt_6M,Short and Term Loans_granted_contracts_amt_6M
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,NaN,NaN,NaN,NaN,20000.0,NaN,NaN
1,001afde5-17dd-48c2-9693-b6df1aa8e752,NaN,NaN,NaN,NaN,9000.0,NaN,NaN
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,NaN,NaN,NaN,5458.0,NaN,NaN,NaN
3,0022f527-0559-43fb-9e2d-65afcffda50b,NaN,NaN,NaN,6562.0,NaN,NaN,NaN
4,003369dc-95be-4643-a498-0ec8898c6837,NaN,NaN,NaN,NaN,8349.0,NaN,NaN


In [57]:
granted_loanamt_by_segment_12m =granted_loans_365days.groupby(['digitalLoanAccountId','loan_segment']).agg(
                                         granted_contracts_amt_12M=('SanctionedAmount', 'sum')).reset_index()


pivot_granted_loanamt_by_segment_12m = granted_loanamt_by_segment_12m.pivot_table(index=['digitalLoanAccountId'], columns='loan_segment', values=['granted_contracts_amt_12M'])

pivot_granted_loanamt_by_segment_12m.columns = [segment+'_granted_contracts_amt_12M' for metric, segment in pivot_granted_loanamt_by_segment_12m.columns]
pivot_granted_loanamt_by_segment_12m = pivot_granted_loanamt_by_segment_12m.reset_index()


pivot_granted_loanamt_by_segment_12m.head(5)

,digitalLoanAccountId,Credit Cards_granted_contracts_amt_12M,Credit Lines_granted_contracts_amt_12M,Home Equity Loans_granted_contracts_amt_12M,Other Loans_granted_contracts_amt_12M,Personal Loans_granted_contracts_amt_12M,Real Estate Loans_granted_contracts_amt_12M,Short and Term Loans_granted_contracts_amt_12M
0,00105a59-1559-4dad-a281-6408eecc96c1,NaN,NaN,NaN,NaN,7999.0,NaN,NaN
1,001a8c7d-b132-4a5b-8686-ebfd945d3bad,NaN,NaN,NaN,NaN,36980.0,NaN,NaN
2,001afde5-17dd-48c2-9693-b6df1aa8e752,NaN,NaN,NaN,NaN,9000.0,NaN,NaN
3,001c77ba-72f9-4c44-a08d-4de9db9f0539,NaN,NaN,NaN,5458.0,NaN,NaN,NaN
4,0022f527-0559-43fb-9e2d-65afcffda50b,NaN,NaN,NaN,6562.0,NaN,NaN,NaN


In [58]:
granted_loansamt_by_segment_24m =granted_loans_730days.groupby(['digitalLoanAccountId','loan_segment']).agg(
                                         granted_contracts_amt_24M=('SanctionedAmount', 'sum')).reset_index()


pivot_granted_loansamt_by_segment_24m = granted_loansamt_by_segment_24m.pivot_table(index=['digitalLoanAccountId'], columns='loan_segment', values=['granted_contracts_amt_24M'])


pivot_granted_loansamt_by_segment_24m.columns = [segment+'_granted_contracts_amt_24M' for metric, segment in pivot_granted_loansamt_by_segment_24m.columns]
pivot_granted_loansamt_by_segment_24m = pivot_granted_loansamt_by_segment_24m.reset_index()


pivot_granted_loansamt_by_segment_24m.head(5)

,digitalLoanAccountId,Credit Cards_granted_contracts_amt_24M,Credit Lines_granted_contracts_amt_24M,Home Equity Loans_granted_contracts_amt_24M,Other Loans_granted_contracts_amt_24M,Personal Loans_granted_contracts_amt_24M,Real Estate Loans_granted_contracts_amt_24M,Short and Term Loans_granted_contracts_amt_24M,Time Loans_granted_contracts_amt_24M,Trust Loans_granted_contracts_amt_24M
0,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,NaN,NaN,67400.0,NaN,NaN,NaN,NaN,NaN
1,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,NaN,NaN,NaN,20000.0,NaN,NaN,NaN,NaN,NaN
2,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,NaN,NaN,NaN,5000.0,NaN,NaN,5000.0,NaN,NaN
3,000d5b99-ffd3-45ad-b650-32fe43a95dc7,140000.0,NaN,NaN,NaN,24959.0,NaN,NaN,NaN,NaN
4,00105a59-1559-4dad-a281-6408eecc96c1,NaN,NaN,NaN,NaN,7999.0,NaN,NaN,NaN,NaN


In [59]:
granted_loan_amt_by_segment = pivot_granted_loanamt_by_segment_6m.merge(pivot_granted_loanamt_by_segment_12m, on=['digitalLoanAccountId'], how='outer')
granted_loan_amt_by_segment = granted_loan_amt_by_segment.merge(pivot_granted_loansamt_by_segment_24m, on=['digitalLoanAccountId'], how='outer')

In [60]:
granted_loan_amt_by_segment.columns

Index(['digitalLoanAccountId', 'Credit Cards_granted_contracts_amt_6M',
       'Credit Lines_granted_contracts_amt_6M',
       'Home Equity Loans_granted_contracts_amt_6M',
       'Other Loans_granted_contracts_amt_6M',
       'Personal Loans_granted_contracts_amt_6M',
       'Real Estate Loans_granted_contracts_amt_6M',
       'Short and Term Loans_granted_contracts_amt_6M',
       'Credit Cards_granted_contracts_amt_12M',
       'Credit Lines_granted_contracts_amt_12M',
       'Home Equity Loans_granted_contracts_amt_12M',
       'Other Loans_granted_contracts_amt_12M',
       'Personal Loans_granted_contracts_amt_12M',
       'Real Estate Loans_granted_contracts_amt_12M',
       'Short and Term Loans_granted_contracts_amt_12M',
       'Credit Cards_granted_contracts_amt_24M',
       'Credit Lines_granted_contracts_amt_24M',
       'Home Equity Loans_granted_contracts_amt_24M',
       'Other Loans_granted_contracts_amt_24M',
       'Personal Loans_granted_contracts_amt_24M',
       '

## Amt of Granted loans 6m,12m,24m

In [61]:
granted_loans_amt_12m =granted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                         tot_granted_contracts_amt_12M=('SanctionedAmount', 'sum')).reset_index()
granted_loans_amt_12m.head()

,digitalLoanAccountId,tot_granted_contracts_amt_12M
0,00105a59-1559-4dad-a281-6408eecc96c1,7999.0
1,001a8c7d-b132-4a5b-8686-ebfd945d3bad,36980.0
2,001afde5-17dd-48c2-9693-b6df1aa8e752,9000.0
3,001c77ba-72f9-4c44-a08d-4de9db9f0539,5458.0
4,0022f527-0559-43fb-9e2d-65afcffda50b,6562.0


In [62]:
granted_loans_amt_6m =granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                         tot_granted_contracts_amt_6M=('SanctionedAmount', 'sum')).reset_index()
granted_loans_amt_6m.head()

,digitalLoanAccountId,tot_granted_contracts_amt_6M
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,20000.0
1,001afde5-17dd-48c2-9693-b6df1aa8e752,9000.0
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,5458.0
3,0022f527-0559-43fb-9e2d-65afcffda50b,6562.0
4,003369dc-95be-4643-a498-0ec8898c6837,8349.0


In [63]:
granted_loans_amt_24m =granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                         tot_granted_contracts_amt_24M=('SanctionedAmount', 'sum')).reset_index()
granted_loans_amt_24m.head()

,digitalLoanAccountId,tot_granted_contracts_amt_24M
0,000a3bee-c57d-4647-beaa-f8d3293b4101,67400.0
1,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,20000.0
2,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,10000.0
3,000d5b99-ffd3-45ad-b650-32fe43a95dc7,164959.0
4,00105a59-1559-4dad-a281-6408eecc96c1,7999.0


In [64]:
granted_loan_amt_df = granted_loans_amt_6m.merge(granted_loans_amt_12m, on=['digitalLoanAccountId'], how='outer')
granted_loan_amt_df = granted_loan_amt_df.merge(granted_loans_amt_24m, on=['digitalLoanAccountId'], how='outer')
granted_loan_amt_df.columns

Index(['digitalLoanAccountId', 'tot_granted_contracts_amt_6M',
       'tot_granted_contracts_amt_12M', 'tot_granted_contracts_amt_24M'],
      dtype='object')

## total, min, max, avg for sanctioned amount

In [65]:
# granted_loans_365days['Sanctioned_amount'] = granted_loans_365days['CreditLimit'].fillna(granted_loans_365days['FinancedAmount'])
grannted_sanctioned_amt_12M= granted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                        avg_amt_granted_12M=('SanctionedAmount', 'mean'),
                                        min_amt_granted_12M=('SanctionedAmount', 'min'),
                                        max_amt_granted_12M=('SanctionedAmount', 'max') ,
#                                         total_amt_granted_12M=('SanctionedAmount', 'sum')
                                        ).reset_index()

grannted_sanctioned_amt_12M.head()

,digitalLoanAccountId,avg_amt_granted_12M,min_amt_granted_12M,max_amt_granted_12M
0,00105a59-1559-4dad-a281-6408eecc96c1,7999.000000,7999.0,7999.0
1,001a8c7d-b132-4a5b-8686-ebfd945d3bad,4108.888889,3980.0,5000.0
2,001afde5-17dd-48c2-9693-b6df1aa8e752,9000.000000,9000.0,9000.0
3,001c77ba-72f9-4c44-a08d-4de9db9f0539,5458.000000,5458.0,5458.0
4,0022f527-0559-43fb-9e2d-65afcffda50b,6562.000000,6562.0,6562.0


In [66]:
grannted_sanctioned_amt_6M= granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                        avg_amt_granted_6M=('SanctionedAmount', 'mean'),
                                        min_amt_granted_6M=('SanctionedAmount', 'min'),
                                        max_amt_granted_6M=('SanctionedAmount', 'max') ,
#                                         total_amt_granted_6M=('SanctionedAmount', 'sum')
                                            ).reset_index()

grannted_sanctioned_amt_6M.head()

,digitalLoanAccountId,avg_amt_granted_6M,min_amt_granted_6M,max_amt_granted_6M
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,4000.0,4000.0,4000.0
1,001afde5-17dd-48c2-9693-b6df1aa8e752,9000.0,9000.0,9000.0
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,5458.0,5458.0,5458.0
3,0022f527-0559-43fb-9e2d-65afcffda50b,6562.0,6562.0,6562.0
4,003369dc-95be-4643-a498-0ec8898c6837,8349.0,8349.0,8349.0


In [67]:
grannted_sanctioned_amt_24M= granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        avg_amt_granted_24M=('SanctionedAmount', 'mean'),
                                        min_amt_granted_24M=('SanctionedAmount', 'min'),
                                        max_amt_granted_24M=('SanctionedAmount', 'max') ,
#                                         total_amt_granted_24M=('SanctionedAmount', 'sum') 
                                            ).reset_index()

grannted_sanctioned_amt_24M.head()

,digitalLoanAccountId,avg_amt_granted_24M,min_amt_granted_24M,max_amt_granted_24M
0,000a3bee-c57d-4647-beaa-f8d3293b4101,67400.0,67400.0,67400.0
1,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,5000.0,5000.0,5000.0
2,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,5000.0,5000.0,5000.0
3,000d5b99-ffd3-45ad-b650-32fe43a95dc7,82479.5,24959.0,140000.0
4,00105a59-1559-4dad-a281-6408eecc96c1,7999.0,7999.0,7999.0


## Avg and Med time difference between granted loans

In [68]:
granted_loan_datediff_12M= granted_loans_365days[['digitalLoanAccountId','ContractStartDate','CBContractCode']]
granted_loan_datediff_12M = granted_loan_datediff_12M.sort_values(by=['digitalLoanAccountId','ContractStartDate','CBContractCode'])


granted_loan_datediff_12M['ContractStartDate'] = pd.to_datetime(granted_loan_datediff_12M['ContractStartDate'])

granted_loan_datediff_12M['day_diff'] = granted_loan_datediff_12M.groupby(['digitalLoanAccountId'])['ContractStartDate'].diff().dt.days


granted_loan_datediff_12M_sorted = granted_loan_datediff_12M.dropna(subset=['day_diff'])

# Sum the differences for each mobile number
granted_loan_datediff_12M_df = granted_loan_datediff_12M_sorted.groupby(['digitalLoanAccountId']).agg(
                                                        no_of_difference=('ContractStartDate', 'count'),
                                                        Total_datediff_12M=('day_diff', 'sum'),
                                                        med_days_bw_contracts_12m=('day_diff', 'median')                                
                                                                ).reset_index()
    
    

# Repayment_cashin.columns = ['mobile_num','no_of_difference', 'Total_cashin_datediff_180days','Median_cashin_datediff_180days','Min_cashin_datediff_180days','Max_cashin_datediff_180days']
granted_loan_datediff_12M_df['avg_days_bw_contracts_12m'] =( granted_loan_datediff_12M_df['Total_datediff_12M']/granted_loan_datediff_12M_df['no_of_difference']).round(2)

granted_loan_datediff_12M_df= granted_loan_datediff_12M_df[['digitalLoanAccountId','med_days_bw_contracts_12m','avg_days_bw_contracts_12m']]

granted_loan_datediff_12M_df.head()

,digitalLoanAccountId,med_days_bw_contracts_12m,avg_days_bw_contracts_12m
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,31.5,39.50
1,004503c8-1892-490c-9838-621d41f61ec2,15.0,15.00
2,0049a771-6757-431e-a7ac-099b3fd5b677,134.0,134.00
3,004fa342-db46-4474-9725-799a841bbf89,26.0,32.33
4,00546418-95f8-4b3b-9518-9c3b523288fd,69.0,69.00


## How many loans were foreclosed? 6m, 12m, 24m

In [69]:

granted_foreclosed_730days = granted_loans_730days[granted_loans_730days['ContractStatusDesc'] == 'Foreclosure']


foreclosure_cnt_24M = granted_foreclosed_730days.groupby(['digitalLoanAccountId']).agg(
                                                            cnt_foreclosed_contracts_24M=('CBContractCode', 'count')).reset_index()


foreclosure_cnt_24M.head()

,digitalLoanAccountId,cnt_foreclosed_contracts_24M
0,00b5bd95-4301-4826-9dc1-d6b4e393de72,1
1,01e321df-0d1e-4b5e-b31a-f58c9ed5cd92,1
2,02181211-ad01-4b98-8743-3a5959f50ab6,1
3,04488834-e5e3-4db2-84d4-d4bcb5a20c8c,1
4,05613b89-d8cb-4367-99f7-fb1d6736c019,1


In [70]:

granted_foreclosed_365days = granted_loans_365days[granted_loans_365days['ContractStatusDesc'] == 'Foreclosure']

foreclosure_cnt_12M = granted_foreclosed_365days.groupby(['digitalLoanAccountId']).agg(
                                                            cnt_foreclosed_contracts_12M=('CBContractCode', 'count')).reset_index()


foreclosure_cnt_12M.head()

,digitalLoanAccountId,cnt_foreclosed_contracts_12M
0,00b5bd95-4301-4826-9dc1-d6b4e393de72,1
1,01e321df-0d1e-4b5e-b31a-f58c9ed5cd92,1
2,02181211-ad01-4b98-8743-3a5959f50ab6,1
3,04488834-e5e3-4db2-84d4-d4bcb5a20c8c,1
4,05613b89-d8cb-4367-99f7-fb1d6736c019,1


In [71]:

granted_foreclosed_180days = granted_loans_180days[granted_loans_180days['ContractStatusDesc'] == 'Foreclosure']

foreclosure_cnt_6M = granted_foreclosed_180days.groupby(['digitalLoanAccountId']).agg(
                                                            cnt_foreclosed_contracts_6M=('CBContractCode', 'count')).reset_index()

foreclosure_cnt_6M.head()

,digitalLoanAccountId,cnt_foreclosed_contracts_6M
0,05613b89-d8cb-4367-99f7-fb1d6736c019,1
1,099d710f-f855-4c8b-b92e-9861b71938af,1
2,0b250219-94fd-4292-b526-ba9a6c658d11,1
3,0cdcd092-4905-4321-a9fb-49d5c7be0afe,1
4,12bd1efc-8303-4796-83cb-a554041154e6,1


## No of written off ever

In [72]:
df[df['ContractStatus'] =='WO']['ContractPhaseDesc'].unique()

array(['Closed', 'Active'], dtype=object)

In [73]:

cnt_writtenoff_ever = granted_loans[granted_loans['ContractStatus'].isin(['WC', 'WF', 'WO'])]


cnt_writtenoff_ever = cnt_writtenoff_ever.groupby(['digitalLoanAccountId']).agg(
                                                            cnt_writeoff_ever=('CBContractCode', 'count')).reset_index()

cnt_writtenoff_ever.head()



,digitalLoanAccountId,cnt_writeoff_ever
0,00e040ab-636b-4813-904d-58666341eba4,1
1,00ff96e4-1a98-44ab-9313-32aff30e451d,1
2,014aac2b-39d9-42f3-9b70-1919db85e7c6,1
3,016d2e0c-192d-4c7c-90d8-08be55cc31a3,2
4,01cd3e54-84ce-4525-9c40-1681fe5e2b61,1


## Velocity loans granted contracts

In [74]:
granted_sanctioned_amt_dfvelocity_24M= granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        avg_amt_granted_24M=('SanctionedAmount', 'mean'),
                                        avg_cnt_granted_24M =('CBContractCode','nunique')).reset_index()

granted_sanctioned_amt_dfvelocity_12M= granted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                        avg_amt_granted_12M=('SanctionedAmount', 'mean'),
                                        avg_cnt_granted_12M =('CBContractCode','nunique')).reset_index()

granted_sanctioned_amt_dfvelocity_6M= granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                        avg_amt_granted_6M=('SanctionedAmount', 'mean'),
                                        avg_cnt_granted_6M =('CBContractCode','nunique')).reset_index()

granted_sanctioned_amt_dfvelocity_3M= granted_loans_90days.groupby(['digitalLoanAccountId']).agg(
                                        avg_amt_granted_3M=('SanctionedAmount', 'mean'),
                                        avg_cnt_granted_3M =('CBContractCode','nunique')).reset_index()


In [75]:
granted_sanctioned_amt_dfvelocity_24M['avg_cnt_granted_24M'] =(granted_sanctioned_amt_dfvelocity_24M['avg_cnt_granted_24M']/24).round(3)
granted_sanctioned_amt_dfvelocity_12M['avg_cnt_granted_12M'] =(granted_sanctioned_amt_dfvelocity_12M['avg_cnt_granted_12M']/12).round(3)
granted_sanctioned_amt_dfvelocity_6M['avg_cnt_granted_6M'] =(granted_sanctioned_amt_dfvelocity_6M['avg_cnt_granted_6M']/6).round(3)
granted_sanctioned_amt_dfvelocity_3M['avg_cnt_granted_3M'] =(granted_sanctioned_amt_dfvelocity_3M['avg_cnt_granted_3M']/3).round(3)

granted_sanctioned_amt_dfvelocity_24M.head()

,digitalLoanAccountId,avg_amt_granted_24M,avg_cnt_granted_24M
0,000a3bee-c57d-4647-beaa-f8d3293b4101,67400.0,0.042
1,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,5000.0,0.083
2,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,5000.0,0.083
3,000d5b99-ffd3-45ad-b650-32fe43a95dc7,82479.5,0.083
4,00105a59-1559-4dad-a281-6408eecc96c1,7999.0,0.042


In [76]:
vel_contract_granted_df = pd.merge(granted_sanctioned_amt_dfvelocity_3M, granted_sanctioned_amt_dfvelocity_6M, on='digitalLoanAccountId', how='outer')
vel_contract_granted_df = pd.merge(vel_contract_granted_df, granted_sanctioned_amt_dfvelocity_12M, on='digitalLoanAccountId', how='outer')
vel_contract_granted_df = pd.merge(vel_contract_granted_df, granted_sanctioned_amt_dfvelocity_24M, on='digitalLoanAccountId', how='outer')
vel_contract_granted_df.head(3)

,digitalLoanAccountId,avg_amt_granted_3M,avg_cnt_granted_3M,avg_amt_granted_6M,avg_cnt_granted_6M,avg_amt_granted_12M,avg_cnt_granted_12M,avg_amt_granted_24M,avg_cnt_granted_24M
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,4000.0,0.667,4000.0,0.833,4108.888889,0.750,4697.500000,0.500
1,0065031d-463d-43ec-802b-082390a75bff,444.0,0.333,1826.6,0.833,6546.454545,0.917,4548.813953,1.792
2,00717397-61f4-429b-b0dd-8db192fa7e46,14789.0,0.333,14789.0,0.167,14789.000000,0.083,14789.000000,0.042


In [77]:
vel_contract_granted_df['vel_contract_granted_cnt_3on12'] = vel_contract_granted_df['avg_cnt_granted_3M']/vel_contract_granted_df['avg_cnt_granted_12M']
vel_contract_granted_df['vel_contract_granted_cnt_6on12'] = vel_contract_granted_df['avg_cnt_granted_6M']/vel_contract_granted_df['avg_cnt_granted_12M']
vel_contract_granted_df['vel_contract_granted_cnt_12on24'] = vel_contract_granted_df['avg_cnt_granted_12M']/vel_contract_granted_df['avg_cnt_granted_24M']

## Velocity loan amount granted contracts

In [78]:
vel_contract_granted_df['vel_contract_granted_amt_3on12'] = vel_contract_granted_df['avg_amt_granted_3M']/vel_contract_granted_df['avg_amt_granted_12M']
vel_contract_granted_df['vel_contract_granted_amt_6on12'] = vel_contract_granted_df['avg_amt_granted_6M']/vel_contract_granted_df['avg_amt_granted_12M']
vel_contract_granted_df['vel_contract_granted_amt_12on24'] = vel_contract_granted_df['avg_amt_granted_12M']/vel_contract_granted_df['avg_amt_granted_24M']

In [79]:
vel_contract_granted_df.columns

Index(['digitalLoanAccountId', 'avg_amt_granted_3M', 'avg_cnt_granted_3M',
       'avg_amt_granted_6M', 'avg_cnt_granted_6M', 'avg_amt_granted_12M',
       'avg_cnt_granted_12M', 'avg_amt_granted_24M', 'avg_cnt_granted_24M',
       'vel_contract_granted_cnt_3on12', 'vel_contract_granted_cnt_6on12',
       'vel_contract_granted_cnt_12on24', 'vel_contract_granted_amt_3on12',
       'vel_contract_granted_amt_6on12', 'vel_contract_granted_amt_12on24'],
      dtype='object')

In [80]:
vel_contract_granted_df = vel_contract_granted_df[['digitalLoanAccountId','vel_contract_granted_cnt_3on12', 'vel_contract_granted_cnt_6on12',
       'vel_contract_granted_cnt_12on24', 'vel_contract_granted_amt_3on12',
       'vel_contract_granted_amt_6on12', 'vel_contract_granted_amt_12on24']]

In [81]:
vel_contract_granted_df.head(2)

,digitalLoanAccountId,vel_contract_granted_cnt_3on12,vel_contract_granted_cnt_6on12,vel_contract_granted_cnt_12on24,vel_contract_granted_amt_3on12,vel_contract_granted_amt_6on12,vel_contract_granted_amt_12on24
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,0.889333,1.110667,1.500000,0.973499,0.973499,0.874697
1,0065031d-463d-43ec-802b-082390a75bff,0.363141,0.908397,0.511719,0.067823,0.279021,1.439156


## Velocity closed amount

In [82]:
closed_granted_loans = df[(df['ContractPhaseDesc']=='Closed') | (df['ContractPhaseDesc']=='Closed in advance') ]

closed_granted_loans['ContractEndDate'] = pd.to_datetime(closed_granted_loans['ContractEndDate'], errors='coerce')
closed_granted_loans['run_date'] = pd.to_datetime(closed_granted_loans['run_date'], errors='coerce')


granted_closed_loans_90days = closed_granted_loans[(closed_granted_loans['ContractEndDate'] >= closed_granted_loans['threshold_90days']) & (closed_granted_loans['ContractEndDate'] <= closed_granted_loans['run_date'])]
granted_closed_loans_180days = closed_granted_loans[(closed_granted_loans['ContractEndDate'] >= closed_granted_loans['threshold_180days']) & (closed_granted_loans['ContractEndDate'] <= closed_granted_loans['run_date'])]
granted_closed_loans_365days = closed_granted_loans[(closed_granted_loans['ContractEndDate'] >= closed_granted_loans['threshold_365days']) & (closed_granted_loans['ContractEndDate'] <= closed_granted_loans['run_date'])]
granted_closed_loans_730days = closed_granted_loans[(closed_granted_loans['ContractEndDate'] >= closed_granted_loans['threshold_730days']) & (closed_granted_loans['ContractEndDate'] <= closed_granted_loans['run_date'])]


In [83]:
# granted_closed_loans_90days = granted_loans_90days[(granted_loans_90days['ContractPhaseDesc']=='Closed') | (granted_loans_90days['ContractPhaseDesc']=='Closed in advance') ]
# granted_closed_loans_180days = granted_loans_180days[(granted_loans_180days['ContractPhaseDesc']=='Closed') | (granted_loans_180days['ContractPhaseDesc']=='Closed in advance') ]
# granted_closed_loans_365days = granted_loans_365days[(granted_loans_365days['ContractPhaseDesc']=='Closed') | (granted_loans_365days['ContractPhaseDesc']=='Closed in advance') ]
# granted_closed_loans_730days = granted_loans_730days[(granted_loans_730days['ContractPhaseDesc']=='Closed') | (granted_loans_730days['ContractPhaseDesc']=='Closed in advance') ]


In [84]:
granted_closed_amt_dfvelocity_24M= granted_closed_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        avg_closed_amt_granted_24M=('SanctionedAmount', 'mean'),
                                        tot_closed_cnt_granted_24M =('CBContractCode','nunique')).reset_index()

granted_closed_amt_dfvelocity_12M= granted_closed_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                        avg_closed_amt_granted_12M=('SanctionedAmount', 'mean'),
                                        tot_closed_cnt_granted_12M =('CBContractCode','nunique')).reset_index()

granted_closed_amt_dfvelocity_6M= granted_closed_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                        avg_closed_amt_granted_6M=('SanctionedAmount', 'mean'),
                                        tot_closed_cnt_granted_6M =('CBContractCode','nunique')).reset_index()

granted_closed_amt_dfvelocity_3M= granted_closed_loans_90days.groupby(['digitalLoanAccountId']).agg(
                                        avg_closed_amt_granted_3M=('SanctionedAmount', 'mean'),
                                        tot_closed_cnt_granted_3M =('CBContractCode','nunique')).reset_index()



In [85]:
granted_closed_amt_dfvelocity_24M['tot_closed_cnt_granted_24M'] =(granted_closed_amt_dfvelocity_24M['tot_closed_cnt_granted_24M']/24).round(3)
granted_closed_amt_dfvelocity_12M['tot_closed_cnt_granted_12M'] =(granted_closed_amt_dfvelocity_12M['tot_closed_cnt_granted_12M']/12).round(3)
granted_closed_amt_dfvelocity_6M['tot_closed_cnt_granted_6M'] =(granted_closed_amt_dfvelocity_6M['tot_closed_cnt_granted_6M']/6).round(3)
granted_closed_amt_dfvelocity_3M['tot_closed_cnt_granted_3M'] =(granted_closed_amt_dfvelocity_3M['tot_closed_cnt_granted_3M']/3).round(3)

granted_closed_amt_dfvelocity_24M.head()

,digitalLoanAccountId,avg_closed_amt_granted_24M,tot_closed_cnt_granted_24M
0,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,5000.0,0.083
1,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,5000.0,0.083
2,000d5b99-ffd3-45ad-b650-32fe43a95dc7,70479.5,0.083
3,00105a59-1559-4dad-a281-6408eecc96c1,7999.0,0.042
4,0010e1ad-59bf-4671-96ef-617668bae133,15000.0,0.042


In [86]:
vel_contract_granted_closed_df = pd.merge(granted_closed_amt_dfvelocity_3M, granted_closed_amt_dfvelocity_6M, on='digitalLoanAccountId', how='outer')
vel_contract_granted_closed_df = pd.merge(vel_contract_granted_closed_df, granted_closed_amt_dfvelocity_12M, on='digitalLoanAccountId', how='outer')
vel_contract_granted_closed_df = pd.merge(vel_contract_granted_closed_df, granted_closed_amt_dfvelocity_24M, on='digitalLoanAccountId', how='outer')
vel_contract_granted_closed_df.head(3)

,digitalLoanAccountId,avg_closed_amt_granted_3M,tot_closed_cnt_granted_3M,avg_closed_amt_granted_6M,tot_closed_cnt_granted_6M,avg_closed_amt_granted_12M,tot_closed_cnt_granted_12M,avg_closed_amt_granted_24M,tot_closed_cnt_granted_24M
0,000d5b99-ffd3-45ad-b650-32fe43a95dc7,24959.0,0.333,70479.5,0.333,70479.5,0.167,70479.5,0.083
1,00105a59-1559-4dad-a281-6408eecc96c1,7999.0,0.333,7999.0,0.167,7999.0,0.083,7999.0,0.042
2,0010e1ad-59bf-4671-96ef-617668bae133,15000.0,0.333,15000.0,0.167,15000.0,0.083,15000.0,0.042


In [87]:
vel_contract_granted_closed_df['vel_contract_closed_amt_3on12'] = vel_contract_granted_closed_df['avg_closed_amt_granted_3M']/vel_contract_granted_closed_df['avg_closed_amt_granted_12M']
vel_contract_granted_closed_df['vel_contract_closed_amt_6on12'] = vel_contract_granted_closed_df['avg_closed_amt_granted_6M']/vel_contract_granted_closed_df['avg_closed_amt_granted_12M']
vel_contract_granted_closed_df['vel_contract_closed_amt_12on24'] = vel_contract_granted_closed_df['avg_closed_amt_granted_12M']/vel_contract_granted_closed_df['avg_closed_amt_granted_24M']

In [88]:
vel_contract_granted_closed_df = vel_contract_granted_closed_df[['digitalLoanAccountId','vel_contract_closed_amt_3on12',
                                                                'vel_contract_closed_amt_6on12','vel_contract_closed_amt_12on24']]
vel_contract_granted_closed_df.head()

,digitalLoanAccountId,vel_contract_closed_amt_3on12,vel_contract_closed_amt_6on12,vel_contract_closed_amt_12on24
0,000d5b99-ffd3-45ad-b650-32fe43a95dc7,0.354131,1.000000,1.000000
1,00105a59-1559-4dad-a281-6408eecc96c1,1.000000,1.000000,1.000000
2,0010e1ad-59bf-4671-96ef-617668bae133,1.000000,1.000000,1.000000
3,001a8c7d-b132-4a5b-8686-ebfd945d3bad,0.844416,0.844416,0.984367
4,001c77ba-72f9-4c44-a08d-4de9db9f0539,1.000000,1.000000,1.000000


## Creditlimit in last 12M

In [89]:
Creditlimit_12m = granted_loans_365days[granted_loans_365days['CreditLimit'].isna() == False]

In [90]:
credit_limit_12mon_df = Creditlimit_12m.groupby(['digitalLoanAccountId']).agg(
                                        avg_credit_limit_12M=('CreditLimit', 'mean'),
                                        min_credit_limit_12M=('CreditLimit', 'min'),
                                        max_credit_limit_12M=('CreditLimit', 'max')).reset_index()

In [91]:
credit_limit_12mon_df.head()

,digitalLoanAccountId,avg_credit_limit_12M,min_credit_limit_12M,max_credit_limit_12M
0,007a6c6f-f038-4547-96c6-2bf778a4cc75,20000.0,20000.0,20000.0
1,00a54bc1-db26-49e7-948c-fa445a0a03e2,55000.0,55000.0,55000.0
2,00c942dc-d45f-4291-ac63-4251138dd56e,20000.0,20000.0,20000.0
3,01ad3f0e-70d0-473c-b9e9-54af0dbe8745,95000.0,95000.0,95000.0
4,01d7ecaa-3bfb-426c-9d07-da0ca8f546aa,35000.0,35000.0,35000.0


## FinancedAmount in last 12M

In [92]:
FinancedAmount_12m = granted_loans_365days[granted_loans_365days['FinancedAmount'].isna() == False]

In [93]:
FinancedAmt_12mon_df = FinancedAmount_12m.groupby(['digitalLoanAccountId']).agg(
                                        avg_financed_amt_12M=('FinancedAmount', 'mean'),
                                        min_financed_amt_12M=('FinancedAmount', 'min'),
                                        max_financed_amt_12M=('FinancedAmount', 'max')).reset_index()

In [94]:
FinancedAmt_12mon_df.head()

,digitalLoanAccountId,avg_financed_amt_12M,min_financed_amt_12M,max_financed_amt_12M
0,00105a59-1559-4dad-a281-6408eecc96c1,7999.000000,7999.0,7999.0
1,001a8c7d-b132-4a5b-8686-ebfd945d3bad,4108.888889,3980.0,5000.0
2,001afde5-17dd-48c2-9693-b6df1aa8e752,9000.000000,9000.0,9000.0
3,001c77ba-72f9-4c44-a08d-4de9db9f0539,5458.000000,5458.0,5458.0
4,0022f527-0559-43fb-9e2d-65afcffda50b,6562.000000,6562.0,6562.0


## Ratio closed loans cnt/ granted loans cnt

In [95]:
total_closed_loans_24M= granted_closed_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        tot_closed_amt_granted_24M=('SanctionedAmount', 'sum'),
                                        tot_closed_cnt_granted_24M =('CBContractCode','count')).reset_index()

total_closed_loans_12M= granted_closed_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                        tot_closed_amt_granted_12M=('SanctionedAmount', 'sum'),
                                        tot_closed_cnt_granted_12M =('CBContractCode','count')).reset_index()

total_closed_loans_6M= granted_closed_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                        tot_closed_amt_granted_6M=('SanctionedAmount', 'sum'),
                                        tot_closed_cnt_granted_6M =('CBContractCode','count')).reset_index()


In [96]:
total_closed_loans_df = total_closed_loans_6M.merge(total_closed_loans_12M, on=['digitalLoanAccountId'],how='outer')
total_closed_loans_df = total_closed_loans_df.merge(total_closed_loans_24M, on=['digitalLoanAccountId'],how='outer')
total_closed_loans_df.head()

,digitalLoanAccountId,tot_closed_amt_granted_6M,tot_closed_cnt_granted_6M,tot_closed_amt_granted_12M,tot_closed_cnt_granted_12M,tot_closed_amt_granted_24M,tot_closed_cnt_granted_24M
0,000d5b99-ffd3-45ad-b650-32fe43a95dc7,140959.0,2.0,140959.0,2.0,140959.0,2
1,00105a59-1559-4dad-a281-6408eecc96c1,7999.0,1.0,7999.0,1.0,7999.0,1
2,0010e1ad-59bf-4671-96ef-617668bae133,15000.0,1.0,15000.0,1.0,15000.0,1
3,001a8c7d-b132-4a5b-8686-ebfd945d3bad,24000.0,6.0,47370.0,10.0,62559.0,13
4,001c77ba-72f9-4c44-a08d-4de9db9f0539,5458.0,1.0,5458.0,1.0,5458.0,1


In [97]:
total_granted_loans_12M= granted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                        total_cnt_granted_12M=('CBContractCode', 'count'),
                                        total_amt_granted_12M=('SanctionedAmount', 'sum') ).reset_index()

total_granted_loans_6M= granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                        total_cnt_granted_6M=('CBContractCode', 'count'),
                                        total_amt_granted_6M=('SanctionedAmount', 'sum') ).reset_index()

total_granted_loans_24M= granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        total_cnt_granted_24M=('CBContractCode', 'count'),
                                        total_amt_granted_24M=('SanctionedAmount', 'sum') ).reset_index()

In [98]:
total_granted_loans_df = total_granted_loans_6M.merge(total_granted_loans_12M, on=['digitalLoanAccountId'],how='outer')
total_granted_loans_df = total_granted_loans_df.merge(total_granted_loans_24M, on=['digitalLoanAccountId'],how='outer')
total_granted_loans_df.head()

,digitalLoanAccountId,total_cnt_granted_6M,total_amt_granted_6M,total_cnt_granted_12M,total_amt_granted_12M,total_cnt_granted_24M,total_amt_granted_24M
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,5.0,20000.0,9.0,36980.0,12,56370.0
1,001afde5-17dd-48c2-9693-b6df1aa8e752,1.0,9000.0,1.0,9000.0,2,16000.0
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,1.0,5458.0,1.0,5458.0,1,5458.0
3,0022f527-0559-43fb-9e2d-65afcffda50b,1.0,6562.0,1.0,6562.0,1,6562.0
4,003369dc-95be-4643-a498-0ec8898c6837,1.0,8349.0,1.0,8349.0,1,8349.0


In [99]:
ratio_btw_closed_granted_loans= total_closed_loans_df.merge(total_granted_loans_df,on=['digitalLoanAccountId'], how='inner')

In [100]:
ratio_btw_closed_granted_loans['ratio_closed_over_new_granted_cnt_6M'] = ratio_btw_closed_granted_loans['tot_closed_cnt_granted_6M']/ratio_btw_closed_granted_loans['total_cnt_granted_6M']
ratio_btw_closed_granted_loans['ratio_closed_over_new_granted_cnt_12M'] = ratio_btw_closed_granted_loans['tot_closed_cnt_granted_12M']/ratio_btw_closed_granted_loans['total_cnt_granted_12M']
ratio_btw_closed_granted_loans['ratio_closed_over_new_granted_cnt_24M'] = ratio_btw_closed_granted_loans['tot_closed_cnt_granted_24M']/ratio_btw_closed_granted_loans['total_cnt_granted_24M']


In [101]:
ratio_btw_closed_granted_loans['ratio_closed_over_new_granted_amt_6M'] = ratio_btw_closed_granted_loans['total_amt_granted_6M']/ratio_btw_closed_granted_loans['tot_closed_amt_granted_6M']
ratio_btw_closed_granted_loans['ratio_closed_over_new_granted_amt_12M'] = ratio_btw_closed_granted_loans['total_amt_granted_12M']/ratio_btw_closed_granted_loans['tot_closed_amt_granted_12M']
ratio_btw_closed_granted_loans['ratio_closed_over_new_granted_amt_24M'] = ratio_btw_closed_granted_loans['total_amt_granted_24M']/ratio_btw_closed_granted_loans['tot_closed_amt_granted_24M']


In [102]:
ratio_btw_closed_granted_loans.columns

Index(['digitalLoanAccountId', 'tot_closed_amt_granted_6M',
       'tot_closed_cnt_granted_6M', 'tot_closed_amt_granted_12M',
       'tot_closed_cnt_granted_12M', 'tot_closed_amt_granted_24M',
       'tot_closed_cnt_granted_24M', 'total_cnt_granted_6M',
       'total_amt_granted_6M', 'total_cnt_granted_12M',
       'total_amt_granted_12M', 'total_cnt_granted_24M',
       'total_amt_granted_24M', 'ratio_closed_over_new_granted_cnt_6M',
       'ratio_closed_over_new_granted_cnt_12M',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_closed_over_new_granted_amt_6M',
       'ratio_closed_over_new_granted_amt_12M',
       'ratio_closed_over_new_granted_amt_24M'],
      dtype='object')

In [103]:
ratio_btw_closed_granted_loans = ratio_btw_closed_granted_loans[['digitalLoanAccountId','ratio_closed_over_new_granted_cnt_6M',
                                                                                   'ratio_closed_over_new_granted_cnt_12M',
                                                                                   'ratio_closed_over_new_granted_cnt_24M',
                                                                                   'ratio_closed_over_new_granted_amt_6M',
                                                                                   'ratio_closed_over_new_granted_amt_12M',
                                                                                   'ratio_closed_over_new_granted_amt_24M']]

In [104]:
ratio_btw_closed_granted_loans.head()

,digitalLoanAccountId,ratio_closed_over_new_granted_cnt_6M,ratio_closed_over_new_granted_cnt_12M,ratio_closed_over_new_granted_cnt_24M,ratio_closed_over_new_granted_amt_6M,ratio_closed_over_new_granted_amt_12M,ratio_closed_over_new_granted_amt_24M
0,000d5b99-ffd3-45ad-b650-32fe43a95dc7,NaN,NaN,1.000000,NaN,NaN,1.170262
1,00105a59-1559-4dad-a281-6408eecc96c1,NaN,1.000000,1.000000,NaN,1.000000,1.000000
2,0010e1ad-59bf-4671-96ef-617668bae133,NaN,NaN,1.000000,NaN,NaN,1.000000
3,001a8c7d-b132-4a5b-8686-ebfd945d3bad,1.2,1.111111,1.083333,0.833333,0.780663,0.901069
4,001c77ba-72f9-4c44-a08d-4de9db9f0539,1.0,1.000000,1.000000,1.000000,1.000000,1.000000


## Ratio of Chargedamount/SanctionedAmount

In [105]:
granted_loans_365days['Ratio_of_ChargedAmout'] = (granted_loans_365days['ChargedAmount']/granted_loans_365days['SanctionedAmount']).round(4)
granted_loans_180days['Ratio_of_ChargedAmout'] = (granted_loans_180days['ChargedAmount']/granted_loans_180days['SanctionedAmount']).round(4)
granted_loans_730days['Ratio_of_ChargedAmout'] = (granted_loans_730days['ChargedAmount']/granted_loans_730days['SanctionedAmount']).round(4)

In [106]:
ratio_charged_on_sanctioned_12M = granted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                        min_ratio_charged_on_sanctioned_12M=('Ratio_of_ChargedAmout', 'min'),
                                        max_ratio_charged_on_sanctioned_12M=('Ratio_of_ChargedAmout', 'max'),
                                        avg_ratio_charged_on_sanctioned_12M=('Ratio_of_ChargedAmout', 'mean')).reset_index()

ratio_charged_on_sanctioned_12M.head()

,digitalLoanAccountId,min_ratio_charged_on_sanctioned_12M,max_ratio_charged_on_sanctioned_12M,avg_ratio_charged_on_sanctioned_12M
0,00105a59-1559-4dad-a281-6408eecc96c1,NaN,NaN,NaN
1,001a8c7d-b132-4a5b-8686-ebfd945d3bad,NaN,NaN,NaN
2,001afde5-17dd-48c2-9693-b6df1aa8e752,NaN,NaN,NaN
3,001c77ba-72f9-4c44-a08d-4de9db9f0539,NaN,NaN,NaN
4,0022f527-0559-43fb-9e2d-65afcffda50b,NaN,NaN,NaN


In [107]:
ratio_charged_on_sanctioned_6M = granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                        min_ratio_charged_on_sanctioned_6M=('Ratio_of_ChargedAmout', 'min'),
                                        max_ratio_charged_on_sanctioned_6M=('Ratio_of_ChargedAmout', 'max'),
                                        avg_ratio_charged_on_sanctioned_6M=('Ratio_of_ChargedAmout', 'mean')).reset_index()

ratio_charged_on_sanctioned_6M.head()

,digitalLoanAccountId,min_ratio_charged_on_sanctioned_6M,max_ratio_charged_on_sanctioned_6M,avg_ratio_charged_on_sanctioned_6M
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,NaN,NaN,NaN
1,001afde5-17dd-48c2-9693-b6df1aa8e752,NaN,NaN,NaN
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,NaN,NaN,NaN
3,0022f527-0559-43fb-9e2d-65afcffda50b,NaN,NaN,NaN
4,003369dc-95be-4643-a498-0ec8898c6837,NaN,NaN,NaN


In [108]:
ratio_charged_on_sanctioned_24M = granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        min_ratio_charged_on_sanctioned_24M=('Ratio_of_ChargedAmout', 'min'),
                                        max_ratio_charged_on_sanctioned_24M=('Ratio_of_ChargedAmout', 'max'),
                                        avg_ratio_charged_on_sanctioned_24M=('Ratio_of_ChargedAmout', 'mean')).reset_index()

ratio_charged_on_sanctioned_24M.head()

,digitalLoanAccountId,min_ratio_charged_on_sanctioned_24M,max_ratio_charged_on_sanctioned_24M,avg_ratio_charged_on_sanctioned_24M
0,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,NaN,NaN
1,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,NaN,NaN,NaN
2,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,NaN,NaN,NaN
3,000d5b99-ffd3-45ad-b650-32fe43a95dc7,NaN,NaN,NaN
4,00105a59-1559-4dad-a281-6408eecc96c1,NaN,NaN,NaN


## Avg, min, max installments

In [109]:
installments_days_6M = granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                        min_no_of_installments_6M=('InstallmentsNumber', 'min'),
                                        max_no_of_installments_6M=('InstallmentsNumber', 'max'),
                                        avg_no_of_installments_6M=('InstallmentsNumber', 'mean')).reset_index()

In [110]:
installments_days_12M = granted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                        min_no_of_installments_12M=('InstallmentsNumber', 'min'),
                                        max_no_of_installments_12M=('InstallmentsNumber', 'max'),
                                        avg_no_of_installments_12M=('InstallmentsNumber', 'mean')).reset_index()

In [111]:
installments_days_24M = granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        min_no_of_installments_24M=('InstallmentsNumber', 'min'),
                                        max_no_of_installments_24M=('InstallmentsNumber', 'max'),
                                        avg_no_of_installments_24M=('InstallmentsNumber', 'mean')).reset_index()

In [112]:
Historical_feat_granted = granted_loan_cnt_by_segment.merge(granted_loan_cnt_df, on ='digitalLoanAccountId', how ='outer')
Historical_feat_granted = Historical_feat_granted.merge(granted_loan_amt_by_segment, on ='digitalLoanAccountId', how ='outer')
Historical_feat_granted = Historical_feat_granted.merge(granted_loan_amt_df, on ='digitalLoanAccountId', how ='outer')
Historical_feat_granted = Historical_feat_granted.merge(grannted_sanctioned_amt_12M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(grannted_sanctioned_amt_24M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(grannted_sanctioned_amt_6M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(granted_loan_datediff_12M_df, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(foreclosure_cnt_6M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(foreclosure_cnt_12M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(foreclosure_cnt_24M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(cnt_writtenoff_ever, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(vel_contract_granted_df, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(vel_contract_granted_closed_df, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(credit_limit_12mon_df, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(FinancedAmt_12mon_df, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(ratio_btw_closed_granted_loans, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(ratio_charged_on_sanctioned_6M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(ratio_charged_on_sanctioned_12M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(ratio_charged_on_sanctioned_24M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(installments_days_6M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(installments_days_12M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(installments_days_24M, on=['digitalLoanAccountId'], how='outer')


In [113]:
Historical_feat_granted.columns

Index(['digitalLoanAccountId', 'Credit Cards_granted_contracts_cnt_6M',
       'Credit Lines_granted_contracts_cnt_6M',
       'Home Equity Loans_granted_contracts_cnt_6M',
       'Other Loans_granted_contracts_cnt_6M',
       'Personal Loans_granted_contracts_cnt_6M',
       'Real Estate Loans_granted_contracts_cnt_6M',
       'Short and Term Loans_granted_contracts_cnt_6M',
       'Credit Cards_granted_contracts_cnt_12M',
       'Credit Lines_granted_contracts_cnt_12M',
       ...
       'avg_ratio_charged_on_sanctioned_24M', 'min_no_of_installments_6M',
       'max_no_of_installments_6M', 'avg_no_of_installments_6M',
       'min_no_of_installments_12M', 'max_no_of_installments_12M',
       'avg_no_of_installments_12M', 'min_no_of_installments_24M',
       'max_no_of_installments_24M', 'avg_no_of_installments_24M'],
      dtype='object', length=107)

# Historical Features for Non Granted 

## Non granted loans cnt and amt

In [114]:
df['ContractRequestDate'] = pd.to_datetime(df['ContractRequestDate']).dt.date

non_granted_loans = df[df['SOURCE']=='nongranted']

nongranted_loans_90days = non_granted_loans[(non_granted_loans['ContractRequestDate'] >= non_granted_loans['threshold_90days']) & (non_granted_loans['ContractRequestDate'] <= non_granted_loans['run_date'])]
nongranted_loans_180days = non_granted_loans[(non_granted_loans['ContractRequestDate'] >= non_granted_loans['threshold_180days']) & (non_granted_loans['ContractRequestDate'] <= non_granted_loans['run_date'])]
nongranted_loans_365days = non_granted_loans[(non_granted_loans['ContractRequestDate'] >= non_granted_loans['threshold_365days']) & (non_granted_loans['ContractRequestDate'] <= non_granted_loans['run_date'])]
nongranted_loans_730days = non_granted_loans[(non_granted_loans['ContractRequestDate'] >= non_granted_loans['threshold_730days']) & (non_granted_loans['ContractRequestDate'] <= non_granted_loans['run_date'])]




In [115]:
nongranted_loans_3M =nongranted_loans_90days.groupby(['digitalLoanAccountId']).agg(
                                         cnt_nongranted_contracts_3M=('CBContractCode', 'nunique'),
                                        amt_nongranted_contracts_3M=('SanctionedAmount', 'sum')).reset_index()

nongranted_loans_3M.head(3)

,digitalLoanAccountId,cnt_nongranted_contracts_3M,amt_nongranted_contracts_3M
0,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,1,0.0
1,00087e49-c94a-482d-831a-8fd762f28ca5,1,12.0
2,000cf231-b6da-4bff-a27e-0880c86c7bbd,1,12.0


In [116]:
nongranted_loans_6M =nongranted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                         cnt_nongranted_contracts_6M=('CBContractCode', 'nunique'),
                                        amt_nongranted_contracts_6M=('SanctionedAmount', 'sum')).reset_index()

nongranted_loans_6M.head(3)

,digitalLoanAccountId,cnt_nongranted_contracts_6M,amt_nongranted_contracts_6M
0,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,1,0.0
1,00087e49-c94a-482d-831a-8fd762f28ca5,1,12.0
2,000cf231-b6da-4bff-a27e-0880c86c7bbd,1,12.0


In [117]:
nongranted_loans_12M =nongranted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                         cnt_nongranted_contracts_12M=('CBContractCode', 'nunique'),
                                        amt_nongranted_contracts_12M=('SanctionedAmount', 'sum')).reset_index()
nongranted_loans_12M.head()

,digitalLoanAccountId,cnt_nongranted_contracts_12M,amt_nongranted_contracts_12M
0,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,1,0.0
1,00087e49-c94a-482d-831a-8fd762f28ca5,1,12.0
2,000a3bee-c57d-4647-beaa-f8d3293b4101,1,0.0
3,000cf231-b6da-4bff-a27e-0880c86c7bbd,1,12.0
4,000e2706-f385-4dab-a434-6ed1792dac06,1,12.0


In [118]:
nongranted_loans_24M =nongranted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                         cnt_nongranted_contracts_24M=('CBContractCode', 'nunique'),
                                        amt_nongranted_contracts_24M=('SanctionedAmount', 'sum')).reset_index()
nongranted_loans_24M.head()

,digitalLoanAccountId,cnt_nongranted_contracts_24M,amt_nongranted_contracts_24M
0,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,1,0.0
1,00087e49-c94a-482d-831a-8fd762f28ca5,1,12.0
2,000a3bee-c57d-4647-beaa-f8d3293b4101,1,0.0
3,000cf231-b6da-4bff-a27e-0880c86c7bbd,1,12.0
4,000e2706-f385-4dab-a434-6ed1792dac06,1,12.0


## Avg and Med time difference between nongranted loans

In [119]:
nongranted_loan_datediff_12M= nongranted_loans_365days[['digitalLoanAccountId','ContractRequestDate','CBContractCode']]
nongranted_loan_datediff_12M = nongranted_loan_datediff_12M.sort_values(by=['digitalLoanAccountId','ContractRequestDate','CBContractCode'])


nongranted_loan_datediff_12M['ContractRequestDate'] = pd.to_datetime(nongranted_loan_datediff_12M['ContractRequestDate'])

nongranted_loan_datediff_12M['day_diff'] = nongranted_loan_datediff_12M.groupby(['digitalLoanAccountId'])['ContractRequestDate'].diff().dt.days


nongranted_loan_datediff_12M_sorted = nongranted_loan_datediff_12M.dropna(subset=['day_diff'])

# Sum the differences for each mobile number
nongranted_loan_datediff_12M_df = nongranted_loan_datediff_12M_sorted.groupby(['digitalLoanAccountId']).agg(
                                                        no_of_difference=('ContractRequestDate', 'count'),
                                                        Tot_days_bw_contracts_12M=('day_diff', 'sum'),
                                                        med_days_bw_contracts_12M=('day_diff', 'median')                                
                                                                ).reset_index()
    
    

# Repayment_cashin.columns = ['mobile_num','no_of_difference', 'Total_cashin_datediff_180days','Median_cashin_datediff_180days','Min_cashin_datediff_180days','Max_cashin_datediff_180days']
nongranted_loan_datediff_12M_df['avg_days_bw_contracts_12M'] =( nongranted_loan_datediff_12M_df['Tot_days_bw_contracts_12M']/nongranted_loan_datediff_12M_df['no_of_difference']).round(2)

nongranted_loan_datediff_12M_df= nongranted_loan_datediff_12M_df[['digitalLoanAccountId','med_days_bw_contracts_12M','avg_days_bw_contracts_12M','Tot_days_bw_contracts_12M']]

nongranted_loan_datediff_12M_df.head()

,digitalLoanAccountId,med_days_bw_contracts_12M,avg_days_bw_contracts_12M,Tot_days_bw_contracts_12M
0,000ef884-bb3d-4e34-80d7-b4f7f1648d23,205.0,205.0,205.0
1,001c77ba-72f9-4c44-a08d-4de9db9f0539,106.0,106.0,106.0
2,004503c8-1892-490c-9838-621d41f61ec2,163.0,163.0,163.0
3,0072658b-6a2c-42ad-b087-3fecef73323d,71.0,86.0,258.0
4,00d44151-63fa-414b-af2f-53106232a74c,173.0,173.0,173.0


## Velocity loans nongranted contracts

In [120]:
nongranted_amt_dfvelocity_24M= nongranted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        avg_amt_nongranted_24M=('SanctionedAmount', 'mean'),
                                        avg_cnt_nongranted_24M =('CBContractCode','nunique')).reset_index()

nongranted_amt_dfvelocity_12M= nongranted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                        avg_amt_nongranted_12M=('SanctionedAmount', 'mean'),
                                        avg_cnt_nongranted_12M =('CBContractCode','nunique')).reset_index()

nongranted_amt_dfvelocity_6M= nongranted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                        avg_amt_nongranted_6M=('SanctionedAmount', 'mean'),
                                        avg_cnt_nongranted_6M =('CBContractCode','nunique')).reset_index()

nongranted_amt_dfvelocity_3M= nongranted_loans_90days.groupby(['digitalLoanAccountId']).agg(
                                        avg_amt_nongranted_3M=('SanctionedAmount', 'mean'),
                                        avg_cnt_nongranted_3M =('CBContractCode','nunique')).reset_index()


In [121]:
nongranted_amt_dfvelocity_24M['avg_cnt_nongranted_24M'] =(nongranted_amt_dfvelocity_24M['avg_cnt_nongranted_24M']/24).round(3)
nongranted_amt_dfvelocity_12M['avg_cnt_nongranted_12M'] =(nongranted_amt_dfvelocity_12M['avg_cnt_nongranted_12M']/12).round(3)
nongranted_amt_dfvelocity_6M['avg_cnt_nongranted_6M'] =(nongranted_amt_dfvelocity_6M['avg_cnt_nongranted_6M']/6).round(3)
nongranted_amt_dfvelocity_3M['avg_cnt_nongranted_3M'] =(nongranted_amt_dfvelocity_3M['avg_cnt_nongranted_3M']/3).round(3)

nongranted_amt_dfvelocity_24M.head()

,digitalLoanAccountId,avg_amt_nongranted_24M,avg_cnt_nongranted_24M
0,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,NaN,0.042
1,00087e49-c94a-482d-831a-8fd762f28ca5,12.0,0.042
2,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,0.042
3,000cf231-b6da-4bff-a27e-0880c86c7bbd,12.0,0.042
4,000e2706-f385-4dab-a434-6ed1792dac06,12.0,0.042


In [122]:
vel_contract_nongranted_df = pd.merge(nongranted_amt_dfvelocity_3M, nongranted_amt_dfvelocity_6M, on='digitalLoanAccountId', how='outer')
vel_contract_nongranted_df = pd.merge(vel_contract_nongranted_df, nongranted_amt_dfvelocity_12M, on='digitalLoanAccountId', how='outer')
vel_contract_nongranted_df = pd.merge(vel_contract_nongranted_df, nongranted_amt_dfvelocity_24M, on='digitalLoanAccountId', how='outer')
vel_contract_nongranted_df.head(3)

,digitalLoanAccountId,avg_amt_nongranted_3M,avg_cnt_nongranted_3M,avg_amt_nongranted_6M,avg_cnt_nongranted_6M,avg_amt_nongranted_12M,avg_cnt_nongranted_12M,avg_amt_nongranted_24M,avg_cnt_nongranted_24M
0,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,NaN,0.333,NaN,0.167,NaN,0.083,NaN,0.042
1,00087e49-c94a-482d-831a-8fd762f28ca5,12.0,0.333,12.0,0.167,12.0,0.083,12.0,0.042
2,000cf231-b6da-4bff-a27e-0880c86c7bbd,12.0,0.333,12.0,0.167,12.0,0.083,12.0,0.042


In [123]:
vel_contract_nongranted_df['vel_contract_nongranted_cnt_3on12'] = vel_contract_nongranted_df['avg_cnt_nongranted_3M']/vel_contract_nongranted_df['avg_cnt_nongranted_12M']
vel_contract_nongranted_df['vel_contract_nongranted_cnt_6on12'] = vel_contract_nongranted_df['avg_cnt_nongranted_6M']/vel_contract_nongranted_df['avg_cnt_nongranted_12M']
vel_contract_nongranted_df['vel_contract_nongranted_cnt_12on24'] = vel_contract_nongranted_df['avg_cnt_nongranted_12M']/vel_contract_nongranted_df['avg_cnt_nongranted_24M']

## Velocity loan amount nongranted contracts

In [124]:
vel_contract_nongranted_df['vel_contract_nongranted_amt_3on12'] = vel_contract_nongranted_df['avg_amt_nongranted_3M']/vel_contract_nongranted_df['avg_amt_nongranted_12M']
vel_contract_nongranted_df['vel_contract_nongranted_amt_6on12'] = vel_contract_nongranted_df['avg_amt_nongranted_6M']/vel_contract_nongranted_df['avg_amt_nongranted_12M']
vel_contract_nongranted_df['vel_contract_nongranted_amt_12on24'] = vel_contract_nongranted_df['avg_amt_nongranted_12M']/vel_contract_nongranted_df['avg_amt_nongranted_24M']

In [125]:
vel_contract_nongranted_df.columns

Index(['digitalLoanAccountId', 'avg_amt_nongranted_3M',
       'avg_cnt_nongranted_3M', 'avg_amt_nongranted_6M',
       'avg_cnt_nongranted_6M', 'avg_amt_nongranted_12M',
       'avg_cnt_nongranted_12M', 'avg_amt_nongranted_24M',
       'avg_cnt_nongranted_24M', 'vel_contract_nongranted_cnt_3on12',
       'vel_contract_nongranted_cnt_6on12',
       'vel_contract_nongranted_cnt_12on24',
       'vel_contract_nongranted_amt_3on12',
       'vel_contract_nongranted_amt_6on12',
       'vel_contract_nongranted_amt_12on24'],
      dtype='object')

In [126]:
vel_contract_nongranted_df = vel_contract_nongranted_df[['digitalLoanAccountId','vel_contract_nongranted_cnt_3on12',
                                                   'vel_contract_nongranted_cnt_6on12',
                                                   'vel_contract_nongranted_cnt_12on24',
                                                   'vel_contract_nongranted_amt_3on12',
                                                   'vel_contract_nongranted_amt_6on12',
                                                   'vel_contract_nongranted_amt_12on24']]

vel_contract_nongranted_df.head()

,digitalLoanAccountId,vel_contract_nongranted_cnt_3on12,vel_contract_nongranted_cnt_6on12,vel_contract_nongranted_cnt_12on24,vel_contract_nongranted_amt_3on12,vel_contract_nongranted_amt_6on12,vel_contract_nongranted_amt_12on24
0,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,4.012048,2.012048,1.976190,NaN,NaN,NaN
1,00087e49-c94a-482d-831a-8fd762f28ca5,4.012048,2.012048,1.976190,1.000000,1.000000,1.0
2,000cf231-b6da-4bff-a27e-0880c86c7bbd,4.012048,2.012048,1.976190,1.000000,1.000000,1.0
3,000e2706-f385-4dab-a434-6ed1792dac06,4.012048,2.012048,1.976190,1.000000,1.000000,1.0
4,000ef884-bb3d-4e34-80d7-b4f7f1648d23,1.994012,1.000000,2.012048,1.993352,1.993352,1.0


In [127]:
Historical_feat_nongranted = nongranted_loans_3M.merge(nongranted_loans_6M, on=['digitalLoanAccountId'], how ='outer')
Historical_feat_nongranted = nongranted_loans_3M.merge(nongranted_loans_12M, on=['digitalLoanAccountId'], how ='outer')
Historical_feat_nongranted = nongranted_loans_3M.merge(nongranted_loans_24M, on=['digitalLoanAccountId'], how ='outer')
Historical_feat_nongranted = nongranted_loans_3M.merge(nongranted_loan_datediff_12M_df, on=['digitalLoanAccountId'], how ='outer')
Historical_feat_nongranted = nongranted_loans_3M.merge(vel_contract_nongranted_df, on=['digitalLoanAccountId'], how ='outer')


# Historical features for Granted and Non Granted Combined

## Ratio of Co-Borrow cnt and Amt

In [128]:
# df['threshold_730days'] = df['run_date'] - pd.Timedelta(days=730)
# df_730days = df[(df['ContractStartDate'] >= df['threshold_730days']) & (df['ContractStartDate'] <= df['run_date'])]

In [129]:

coborrow_df_24M = granted_loans_730days[granted_loans_730days['RoleDesc'] =='Co-Borrower']

coborrow_ratio_f_24M = coborrow_df_24M.groupby(['digitalLoanAccountId']).agg(
                                        cnt_contract_coborrower_24M =('CBContractCode', 'count'),
                                        amt_contract_coborrower_24M=('SanctionedAmount', 'sum'),
                                            ).reset_index()



coborrow_ratio_f_24M.head(5)


,digitalLoanAccountId,cnt_contract_coborrower_24M,amt_contract_coborrower_24M
0,0092cf71-1ed9-4035-819a-139f7571d5cf,1,1084000.0
1,02357c5b-fe46-4eef-b0ac-d087b81f4c91,1,1176000.0
2,05134374-c2c5-483c-8d73-6c33f5ac74c8,2,40000.0
3,0586eaf8-2a59-418b-a732-badd0190aa08,1,1879200.0
4,063fb8a0-7dc2-4c4d-b4e7-11fe1124a9ec,1,776000.0


In [130]:

grantedloans_ratio_f_24M = granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        cnt_granted_loans_24M =('CBContractCode', 'count'),
                                        amt_granted_loans_24M=('SanctionedAmount', 'sum'),
                                            ).reset_index()



grantedloans_ratio_f_24M.head(5)


,digitalLoanAccountId,cnt_granted_loans_24M,amt_granted_loans_24M
0,000a3bee-c57d-4647-beaa-f8d3293b4101,1,67400.0
1,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,4,20000.0
2,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,2,10000.0
3,000d5b99-ffd3-45ad-b650-32fe43a95dc7,2,164959.0
4,00105a59-1559-4dad-a281-6408eecc96c1,1,7999.0


In [131]:
ratio_coborrow_vs_grantedloans = coborrow_ratio_f_24M.merge(grantedloans_ratio_f_24M, on='digitalLoanAccountId')
ratio_coborrow_vs_grantedloans['ratio_coborrow_contracts_cnt_24M'] =  ratio_coborrow_vs_grantedloans['cnt_contract_coborrower_24M']/ratio_coborrow_vs_grantedloans['cnt_granted_loans_24M']
ratio_coborrow_vs_grantedloans['ratio_coborrow_contracts_amt_24M'] =  ratio_coborrow_vs_grantedloans['amt_contract_coborrower_24M']/ratio_coborrow_vs_grantedloans['amt_granted_loans_24M']

ratio_coborrow_vs_grantedloans = ratio_coborrow_vs_grantedloans[['digitalLoanAccountId','ratio_coborrow_contracts_cnt_24M','ratio_coborrow_contracts_amt_24M']]

## applicant_name_match_score

In [132]:
from fuzzywuzzy import fuzz
# Handle missing values by filling them with empty strings
df['Firstname'] = df['Firstname'].fillna('')
df['middleName'] = df['middleName'].fillna('')
df['LastName'] = df['LastName'].fillna('')
df['LinkedSubject_Name'] = df['LinkedSubject_Name'].fillna('')
 
# Create the FullName column by concatenating Firstname, middleName, and LastName
df['FullName'] = df['Firstname'] + ' ' + df['middleName'] + ' ' + df['LastName']
 
# Ensure all values are strings
df['FullName'] = df['FullName'].astype(str)
df['LinkedSubject_Name'] = df['LinkedSubject_Name'].astype(str)
 
# Function to calculate fuzzy match score with set matching
def calculate_match_score(name1, name2):
    # If either name is empty, return -9
    if not name1.strip() or not name2.strip():
        return -9
    return fuzz.token_set_ratio(name1, name2)
 
# Apply the function to calculate match scores for FullName vs LinkedSubject_Name
df['applicant_name_match_score'] = df.apply(lambda row: calculate_match_score(row['FullName'], row['LinkedSubject_Name']), axis=1)

In [133]:
applicant_name_match_score = df[['digitalLoanAccountId', 'applicant_name_match_score']].drop_duplicates(keep='first')
applicant_name_match_score.head()

,digitalLoanAccountId,applicant_name_match_score
1,001afde5-17dd-48c2-9693-b6df1aa8e752,-9
2,0028d7ec-e8b9-45dc-b39c-e0f4241c4f01,-9
3,00546418-95f8-4b3b-9518-9c3b523288fd,-9
4,006b0eb3-c728-485b-b01d-1d1a84b00562,-9
7,0092cf71-1ed9-4035-819a-139f7571d5cf,-9


## applicant_employment_match_score

In [134]:
from fuzzywuzzy import fuzz
# Handle missing values by filling them with empty strings
df['natureofwork'] = df['natureofwork'].fillna('')
df['PSICDesc'] = df['PSICDesc'].fillna('')
 
# Ensure all values are strings
df['natureofwork'] = df['natureofwork'].astype(str)
df['PSICDesc'] = df['PSICDesc'].astype(str)
 
# Function to calculate fuzzy match score with set matching
def calculate_match_score(name1, name2):
    # If either name is empty, return -9
    if not name1.strip() or not name2.strip():
        return -9
    return fuzz.token_set_ratio(name1, name2)
 
# Apply the function to calculate match scores
df['applicant_employment_match_score'] = df.apply(lambda row: calculate_match_score(row['natureofwork'], row['PSICDesc']), axis=1)
 
applicant_employment_match_score = df[['digitalLoanAccountId','applicant_employment_match_score']].drop_duplicates(keep='first')

In [135]:
applicant_employment_match_score.head()

,digitalLoanAccountId,applicant_employment_match_score
1,001afde5-17dd-48c2-9693-b6df1aa8e752,-9
2,0028d7ec-e8b9-45dc-b39c-e0f4241c4f01,-9
3,00546418-95f8-4b3b-9518-9c3b523288fd,-9
4,006b0eb3-c728-485b-b01d-1d1a84b00562,-9
7,0092cf71-1ed9-4035-819a-139f7571d5cf,-9


In [136]:
import re
# Fill NAs and concatenate fields
df['subIndustryDescription'] = df['subIndustryDescription'].fillna(' ').astype(str)
df['industryDescription'] = df['industryDescription'].fillna(' ').astype(str)
df['OccupationDesc'] = df['OccupationDesc'].fillna(' ').astype(str)
df['OccupationStatusDesc'] = df['OccupationStatusDesc'].fillna(' ').astype(str)
 
df['industryDerived'] = df['subIndustryDescription'] + ' ' + df['industryDescription']
df['occupationDerived'] = df['OccupationDesc'] + ' ' + df['OccupationStatusDesc']
 
# Remove numbers from strings
def remove_numbers(s):
    return re.sub(r'\d+', '', s)
 
df['industryDerived'] = df['industryDerived'].apply(remove_numbers)
df['occupationDerived'] = df['occupationDerived'].apply(remove_numbers)
 
# Function to calculate fuzzy match score
def calculate_match_score(name1, name2):
    if not name1.strip() or not name2.strip():
        return -9
    return fuzz.token_set_ratio(name1, name2)
 
# Apply function to calculate match scores
df['industry_employment_match_score'] = df.apply(
    lambda row: calculate_match_score(row['industryDerived'], row['occupationDerived']),
    axis=1
)
 
# Merge the results with the existing DataFrame
applicant_employment_match_score = pd.merge(
    applicant_employment_match_score,
    df[['digitalLoanAccountId', 'industry_employment_match_score']],
    on='digitalLoanAccountId',
    how='left'
)
applicant_employment_match_score = applicant_employment_match_score.drop_duplicates()
 
# Display the updated DataFrame
applicant_employment_match_score.head()

,digitalLoanAccountId,applicant_employment_match_score,industry_employment_match_score
0,001afde5-17dd-48c2-9693-b6df1aa8e752,-9,-9
2,0028d7ec-e8b9-45dc-b39c-e0f4241c4f01,-9,52
10,00546418-95f8-4b3b-9518-9c3b523288fd,-9,-9
13,006b0eb3-c728-485b-b01d-1d1a84b00562,-9,35
21,0092cf71-1ed9-4035-819a-139f7571d5cf,-9,22


In [137]:
Historical_feat_combined = ratio_coborrow_vs_grantedloans.merge(applicant_employment_match_score, on=['digitalLoanAccountId'], how ='outer')
Historical_feat_combined = Historical_feat_combined.merge(applicant_name_match_score, on=['digitalLoanAccountId'], how ='outer')

# Scorecard Features Binning

In [138]:
df_CreditMaxOverdueDays = df.groupby('digitalLoanAccountId')['CreditMaxOverdueDays'].max().reset_index()


df_CreditMaxOverdueDays['CreditMaxOverdueDays_0123'] = np.where(df_CreditMaxOverdueDays['CreditMaxOverdueDays'].isin([0, 1, 2, 3]), 1, 0)
df_CreditMaxOverdueDays['CreditMaxOverdueDays_456'] = np.where(df_CreditMaxOverdueDays['CreditMaxOverdueDays'].isin([4,5,6]), 1, 0)
df_CreditMaxOverdueDays['CreditMaxOverdueDays_other'] = np.where(~df_CreditMaxOverdueDays['CreditMaxOverdueDays'].isin([0,1,2,3,4,5,6]), 1, 0)


df_CreditMaxOverdueDays = df_CreditMaxOverdueDays[['digitalLoanAccountId','CreditMaxOverdueDays_0123','CreditMaxOverdueDays_456','CreditMaxOverdueDays_other']]
df_CreditMaxOverdueDays.head()

,digitalLoanAccountId,CreditMaxOverdueDays_0123,CreditMaxOverdueDays_456,CreditMaxOverdueDays_other
0,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,0,0,1
1,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,0,0,1
2,00087e49-c94a-482d-831a-8fd762f28ca5,0,0,1
3,000a3bee-c57d-4647-beaa-f8d3293b4101,0,0,1
4,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,0,0,1


In [139]:
df_CreditAvgCreditLimit = df.groupby('digitalLoanAccountId')['CreditAvgCreditLimit'].max().reset_index()

df_CreditAvgCreditLimit['CreditAvgCreditLimit_<2k'] = np.where(df_CreditAvgCreditLimit['CreditAvgCreditLimit'] < 2000, 1, 0)
df_CreditAvgCreditLimit['CreditAvgCreditLimit_2k_to_42k'] = np.where((df_CreditAvgCreditLimit['CreditAvgCreditLimit'] >= 2000) & (df_CreditAvgCreditLimit['CreditAvgCreditLimit'] < 42000), 1, 0)
df_CreditAvgCreditLimit['CreditAvgCreditLimit_42k_to_106k'] = np.where((df_CreditAvgCreditLimit['CreditAvgCreditLimit'] >= 42000) & (df_CreditAvgCreditLimit['CreditAvgCreditLimit'] < 106000), 1, 0)
df_CreditAvgCreditLimit['CreditAvgCreditLimit_>=106k'] = np.where(df_CreditAvgCreditLimit['CreditAvgCreditLimit'] >= 106000, 1, 0)
df_CreditAvgCreditLimit['CreditAvgCreditLimit_other'] = np.where(df_CreditAvgCreditLimit['CreditAvgCreditLimit'].isna(), 1, 0)

df_CreditAvgCreditLimit = df_CreditAvgCreditLimit[['digitalLoanAccountId',
                                           'CreditAvgCreditLimit_<2k', 'CreditAvgCreditLimit_2k_to_42k',
                                           'CreditAvgCreditLimit_42k_to_106k', 'CreditAvgCreditLimit_>=106k',
                                           'CreditAvgCreditLimit_other']]
df_CreditAvgCreditLimit.head()

,digitalLoanAccountId,CreditAvgCreditLimit_<2k,CreditAvgCreditLimit_2k_to_42k,CreditAvgCreditLimit_42k_to_106k,CreditAvgCreditLimit_>=106k,CreditAvgCreditLimit_other
0,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,1,0,0,0,0
1,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,1,0,0,0,0
2,00087e49-c94a-482d-831a-8fd762f28ca5,1,0,0,0,0
3,000a3bee-c57d-4647-beaa-f8d3293b4101,1,0,0,0,0
4,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,1,0,0,0,0


In [140]:
# Create the new DataFrame with specified columns and filter
df_accountsOverduePct = df[df['ContractPhaseDesc'] == 'Active'][[
    'digitalLoanAccountId',
    'customerId',
    'ContractPhaseDesc',
    'ContractStartDate',
    'ContractEndDate',
    'OverdueDaysDesc',
    'CBContractCode'
]]
 
# Define the categories of overdue days to count
overdue_categories = [
    '91-180 days delay / More than 3 Cycles late',
    '181-365 days delay',
    'More than 1 year delay'
]

df_filtered_overdue = df_accountsOverduePct[df_accountsOverduePct['OverdueDaysDesc'].isin(overdue_categories)]

#Group by 'digitalLoanAccountId' and count occurrences
df_count_90days = df_filtered_overdue.groupby('digitalLoanAccountId').size().reset_index(name='accounts90DaysCount')

 
# # Create the 'accounts90DaysCount' column that counts the number of rows per 'digitalLoanAccountId' for the specified overdue categories
# df_accountsOverduePct['accounts90DaysCount'] = df_accountsOverduePct[df_accountsOverduePct['OverdueDaysDesc'].isin(overdue_categories)].groupby('digitalLoanAccountId')['OverdueDaysDesc'].transform('count')
 

# # Create the 'total_active' column that counts the number of rows per 'digitalLoanAccountId'
df_accountsOverduePct_active = df_accountsOverduePct.groupby('digitalLoanAccountId').agg(
                                        total_active =('CBContractCode', 'count')).reset_index()
 
#Merge the counts back into the original DataFrame
df_accountsOverduePct = df_accountsOverduePct_active.merge(df_count_90days, on='digitalLoanAccountId', how='left')
    
# # Calculate 'accountsOverduePct'
df_accountsOverduePct['accountsOverduePct'] = df_accountsOverduePct['accounts90DaysCount'] / df_accountsOverduePct['total_active'] * 100
 
df_accountsOverduePct_ = df_accountsOverduePct

In [141]:
# # Create new columns based on 'accountsOverduePct'
df_accountsOverduePct['NumberOverduePct<17'] = np.where(df_accountsOverduePct['accountsOverduePct'] < 17, 1, 0)
df_accountsOverduePct['NumberOverduePct<41'] = np.where((df_accountsOverduePct['accountsOverduePct'] >= 17) & (df_accountsOverduePct['accountsOverduePct'] < 41), 1, 0)
df_accountsOverduePct['NumberOverduePct>=41'] = np.where(df_accountsOverduePct['accountsOverduePct'] >= 41, 1, 0)
df_accountsOverduePct['NumberOverduePct_other'] = np.where(df_accountsOverduePct['accountsOverduePct'].isna(), 1, 0)
 
    
df_accountsOverduePct = df_accountsOverduePct[['digitalLoanAccountId', 'NumberOverduePct<17', 'NumberOverduePct<41',
                       'NumberOverduePct>=41', 'NumberOverduePct_other']]

# # Display the new DataFrame
df_accountsOverduePct.head()

,digitalLoanAccountId,NumberOverduePct<17,NumberOverduePct<41,NumberOverduePct>=41,NumberOverduePct_other
0,000a3bee-c57d-4647-beaa-f8d3293b4101,0,0,0,1
1,000d5b99-ffd3-45ad-b650-32fe43a95dc7,0,0,0,1
2,001427de-8ac2-4a4e-aabb-40942c743276,0,0,0,1
3,001afde5-17dd-48c2-9693-b6df1aa8e752,0,0,0,1
4,0021596e-dcda-4634-838f-9475aef54834,0,0,0,1


In [142]:
# Group by 'digitalLoanAccountId' and get the maximum 'NonInstAvgCreditLimit'
df_NonInstAvgCreditLimit = df.groupby('digitalLoanAccountId')['NonInstAvgCreditLimit'].max().reset_index()
 
# Apply conditions to create new columns based on 'NonInstAvgCreditLimit'
df_NonInstAvgCreditLimit['NonInstAvgCreditLimit<15k'] = np.where(df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] < 15000, 1, 0)
df_NonInstAvgCreditLimit['NonInstAvgCreditLimit<35k'] = np.where((df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] >= 15000) & (df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] < 35000), 1, 0)
df_NonInstAvgCreditLimit['NonInstAvgCreditLimit<65k'] = np.where((df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] >= 35000) & (df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] < 65000), 1, 0)
df_NonInstAvgCreditLimit['NonInstAvgCreditLimit>=65k'] = np.where((df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] >= 65000) & (df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] < 106000), 1, 0)
df_NonInstAvgCreditLimit['NonInstAvgCreditLimit>=106k'] = np.where(df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] >= 106000, 1, 0)
 # Handle NaN values in 'NonInstAvgCreditLimit' (if applicable) by using the same DataFrame
df_NonInstAvgCreditLimit['NonInstAvgCreditLimit_others'] = np.where(df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'].isna(), 1, 0)

    
df_NonInstAvgCreditLimit= df_NonInstAvgCreditLimit[['digitalLoanAccountId','NonInstAvgCreditLimit<15k','NonInstAvgCreditLimit<35k', 'NonInstAvgCreditLimit<65k','NonInstAvgCreditLimit>=65k','NonInstAvgCreditLimit>=106k','NonInstAvgCreditLimit_others']]
 
df_NonInstAvgCreditLimit.head()

,digitalLoanAccountId,NonInstAvgCreditLimit<15k,NonInstAvgCreditLimit<35k,NonInstAvgCreditLimit<65k,NonInstAvgCreditLimit>=65k,NonInstAvgCreditLimit>=106k,NonInstAvgCreditLimit_others
0,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,1,0,0,0,0,0
1,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,1,0,0,0,0,0
2,00087e49-c94a-482d-831a-8fd762f28ca5,1,0,0,0,0,0
3,000a3bee-c57d-4647-beaa-f8d3293b4101,1,0,0,0,0,0
4,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,1,0,0,0,0,0


In [143]:
df_ScoreRange = df.groupby('digitalLoanAccountId')['ScoreRange'].max().reset_index()

df_ScoreRange['ScoreRange_HiJiDiEiFiGiIi'] = np.where(df_ScoreRange['ScoreRange'].isin(['Hi','Ji','Di','Ei','Fi','Gi','Ii']), 1, 0)
df_ScoreRange['ScoreRange_Ci'] = np.where(df_ScoreRange['ScoreRange'].isin(['Ci']), 1, 0)
df_ScoreRange['ScoreRange_Bi'] = np.where(df_ScoreRange['ScoreRange'].isin(['Bi']), 1, 0)
df_ScoreRange['ScoreRange_Other'] = np.where(~df_ScoreRange['ScoreRange'].isin(['Hi','Ji','Di','Ei','Fi','Gi','Ii','Ci','Bi']), 1, 0)
df_ScoreRange = df_ScoreRange[['digitalLoanAccountId','ScoreRange_HiJiDiEiFiGiIi','ScoreRange_Ci','ScoreRange_Bi','ScoreRange_Other']]
df_ScoreRange.head()

,digitalLoanAccountId,ScoreRange_HiJiDiEiFiGiIi,ScoreRange_Ci,ScoreRange_Bi,ScoreRange_Other
0,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,0,0,0,1
1,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,0,0,0,1
2,00087e49-c94a-482d-831a-8fd762f28ca5,0,0,0,1
3,000a3bee-c57d-4647-beaa-f8d3293b4101,1,0,0,0
4,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,1,0,0,0


In [144]:
ScoreCard_Features = df_CreditMaxOverdueDays.merge(df_CreditAvgCreditLimit, on='digitalLoanAccountId',how ='outer')
ScoreCard_Features = ScoreCard_Features.merge(df_accountsOverduePct_, on='digitalLoanAccountId',how ='outer')
ScoreCard_Features = ScoreCard_Features.merge(df_NonInstAvgCreditLimit, on='digitalLoanAccountId',how ='outer')
ScoreCard_Features = ScoreCard_Features.merge(df_ScoreRange, on='digitalLoanAccountId',how ='outer')

# Scorecard Feat for Master Table

In [145]:
CreditMaxOverdueDays = df[['digitalLoanAccountId','CreditMaxOverdueDays']].drop_duplicates(keep='first')
CreditAvgCreditLimit = df[['digitalLoanAccountId','CreditAvgCreditLimit']].drop_duplicates(keep='first')
ScoreRange = df[['digitalLoanAccountId','ScoreRange']].drop_duplicates(keep='first')
NonInstAvgCreditLimit = df[['digitalLoanAccountId','NonInstAvgCreditLimit']].drop_duplicates(keep='first')
accountsOverduePct = df_accountsOverduePct_[['digitalLoanAccountId','accountsOverduePct']].drop_duplicates(keep='first')

In [146]:
ScoreCard_Features_Master = CreditMaxOverdueDays.merge(CreditAvgCreditLimit, on='digitalLoanAccountId',how ='outer')
ScoreCard_Features_Master = ScoreCard_Features_Master.merge(accountsOverduePct, on='digitalLoanAccountId',how ='outer')
ScoreCard_Features_Master = ScoreCard_Features_Master.merge(NonInstAvgCreditLimit, on='digitalLoanAccountId',how ='outer')
ScoreCard_Features_Master = ScoreCard_Features_Master.merge(ScoreRange, on='digitalLoanAccountId',how ='outer')

# Master table

In [147]:
Master_table = df_table_A.merge(snapshot_feature_df,on=['digitalLoanAccountId'],how='left')
Master_table = Master_table.merge(Historical_feat_granted,on=['digitalLoanAccountId'],how='left')
Master_table = Master_table.merge(Historical_feat_nongranted,on=['digitalLoanAccountId'],how='left')
Master_table = Master_table.merge(Historical_feat_combined,on=['digitalLoanAccountId'],how='left')
Master_table = Master_table.merge(ScoreCard_Features_Master,on=['digitalLoanAccountId'],how='left')

In [148]:
Master_table.head()

,digitalLoanAccountId,customerId,disbursementDateTime,targetdataselection_SIL,defFPD30,cnt_active_contracts,cnt_active_contracts_Credit Cards,cnt_active_contracts_Credit Lines,cnt_active_contracts_Home Equity Loans,cnt_active_contracts_Other Loans,...,ratio_coborrow_contracts_cnt_24M,ratio_coborrow_contracts_amt_24M,applicant_employment_match_score,industry_employment_match_score,applicant_name_match_score,CreditMaxOverdueDays,CreditAvgCreditLimit,accountsOverduePct,NonInstAvgCreditLimit,ScoreRange
0,001afde5-17dd-48c2-9693-b6df1aa8e752,2553421,2024-06-05 15:49:19,Test,0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,-9,-9,-9,NaN,0,NaN,0,Gi
1,0028d7ec-e8b9-45dc-b39c-e0f4241c4f01,2630227,2024-07-03 14:21:13,Other,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-9,52,-9,NaN,0,NaN,0,NaN
2,0028d7ec-e8b9-45dc-b39c-e0f4241c4f01,2630227,2024-07-03 14:21:13,Other,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-9,52,-9,NaN,0,NaN,0,NaN
3,0028d7ec-e8b9-45dc-b39c-e0f4241c4f01,2630227,2024-07-03 14:21:13,Other,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-9,52,-9,NaN,0,NaN,0,NaN
4,0028d7ec-e8b9-45dc-b39c-e0f4241c4f01,2630227,2024-07-03 14:21:13,Other,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-9,52,-9,NaN,0,NaN,0,NaN


In [149]:
Master_table.columns.nunique()

165

In [150]:
def replace_spaces_in_columns(df):
    # Replace spaces with underscores in column names
    df.columns = df.columns.str.replace(' ', '_')
    return df

In [151]:
Master_table = replace_spaces_in_columns(Master_table)

In [152]:
Master_table.columns.tolist()

['digitalLoanAccountId',
 'customerId',
 'disbursementDateTime',
 'targetdataselection_SIL',
 'defFPD30',
 'cnt_active_contracts',
 'cnt_active_contracts_Credit_Cards',
 'cnt_active_contracts_Credit_Lines',
 'cnt_active_contracts_Home_Equity_Loans',
 'cnt_active_contracts_Other_Loans',
 'cnt_active_contracts_Personal_Loans',
 'cnt_active_contracts_Real_Estate_Loans',
 'cnt_active_contracts_Short_and_Term_Loans',
 'cnt_active_contracts_Time_Loans',
 'max_amt_active_contracts',
 'tot_active_contracts_os_amt',
 'tot_active_contracts_util',
 'Credit_Cards_active_contracts_util',
 'Credit_Lines_active_contracts_util',
 'Home_Equity_Loans_active_contracts_util',
 'Other_Loans_active_contracts_util',
 'Personal_Loans_active_contracts_util',
 'Real_Estate_Loans_active_contracts_util',
 'Short_and_Term_Loans_active_contracts_util',
 'Time_Loans_active_contracts_util',
 'last_repay_amt_30d',
 'highest_latest_repay_method_30d',
 'last_repay_amt_60d',
 'highest_latest_repay_method_60d',
 'last_rep

In [153]:
Master_table.to_csv('Master_table_SIL.csv',index=False)

In [154]:
# Master_table_scorecard = df_table_A.merge(ScoreCard_Features,on=['digitalLoanAccountId'],how='left')
# Master_table_scorecard.head()

In [155]:
# Master_table_scorecard.to_csv('Master_table_Scorecard.csv')

In [156]:
# len(Master_table_scorecard)